In [ ]:
!pip install --quiet openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


In [ ]:
import warnings
import os
import pandas as pd
from google.colab import drive
import pickle
import numpy as np
from openai import *
import logging
import re

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title get_sql_tag
def get_sql_query_level(sql_query):
    # Function to classify SQL query and return just the level tag
    def classify_sql_query_level(sql_query):
        sql_query_lower = sql_query.lower()
        level = 6  # Default to OTHER

        if "select" in sql_query_lower and "from" in sql_query_lower:
            level = 0  # SELECT
            if "max(" in sql_query_lower or "min(" in sql_query_lower or \
               "avg(" in sql_query_lower or "count(" in sql_query_lower or \
               "sum(" in sql_query_lower:
                level = 1  # SIMPLE_AGGR

        if "where" in sql_query_lower:
            level = 2  # WHERE

        if "group by" in sql_query_lower:
            level = 3  # GROUPBY

        if "having" in sql_query_lower:
            level = 4  # HAVING

        if "order by" in sql_query_lower:
            level = 5  # ORDERBY

        return level

    # Classification tags
    classification_tags = {
        0: "SELECT",
        1: "SIMPLE_AGGR",
        2: "WHERE",
        3: "GROUPBY",
        4: "HAVING",
        5: "ORDERBY",
        6: "OTHER"
    }

    # Classify the SQL query
    level = classify_sql_query_level(sql_query)

    # Return the classification tag
    return classification_tags[level]

In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
with open('/content/drive/MyDrive/ProprietaryDatasetsDict.pkl', 'rb') as f:
      db_to_df_proprietary = pickle.load(f)

qatch_pickle = pd.read_pickle('/content/drive/MyDrive/spider/QATCH_proprietary_v2.pkl')

In [ ]:
##get the schema for teh proprietary tables
db_to_df_proprietary["breastCancer"].columns

Index(['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status'],
      dtype='object')

In [ ]:
##get teh schema for the proprietary tables
db_to_df_proprietary["heartAttack"].columns

Index(['age', 'sex', 'chestpaintype', 'restingbloodpressure',
       'cholestoralinmg', 'fastingbloodsugar',
       'restingelectrocardiographicrresults', 'numberofmajorvvessels', 'thall',
       'output'],
      dtype='object')

In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
ProprietaryDomains=['medicine','miscellaneous','ecommerce','finance']
qatch_pickle=qatch_pickle[qatch_pickle['domain']==ProprietaryDomains[0]]

In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
qatch_pickle['sql_tags'].unique()

array(['SELECT-ALL', 'SELECT-ADD-COL', 'SELECT-RANDOM-COL',
       'ORDERBY-SINGLE', 'ORDERBY-PROJECT', 'DISTINCT-SINGLE',
       'DISTINCT-MULT', 'WHERE-CAT-MOST-FREQUENT',
       'WHERE-CAT-LEAST-FREQUENT', 'WHERE-NOT-MOST-FREQUENT',
       'WHERE-NOT-LEAST-FREQUENT', 'WHERE-NUM-MAX-VALUES',
       'WHERE-NUM-MIN-VALUES', 'WHERE-NUM-MEAN-VALUES', 'GROUPBY-NO-AGGR',
       'GROUPBY-AGG-MIN', 'GROUPBY-AGG-MAX', 'GROUPBY-COUNT',
       'GROUPBY-AGG-SUM', 'HAVING-COUNT-GR', 'HAVING-COUNT-LS',
       'HAVING-AGG-AVG-GR', 'HAVING-AGG-AVG-LS', 'HAVING-AGG-SUM-GR',
       'HAVING-AGG-SUM-LS', 'HAVING-COUNT-EQ',
       'SIMPLE-AGG-COUNT-DISTINCT', 'SIMPLE-AGG-MAX', 'SIMPLE-AGG-MIN'],
      dtype=object)

In [ ]:
# db_save_path_spider = '/content/drive/MyDrive/spider/test_database'
# databasesSpider = MultipleDatabases(db_save_path_spider)
# evaluator_Spider = MetricEvaluator(databases=databasesSpider)

with open('/content/drive/MyDrive/spider/db_table_to_df_valid.pkl', 'rb') as file:
    db_to_df_spiderTest = pickle.load(file)

with open('/content/drive/MyDrive/spider/db_table_to_df.pkl', 'rb') as file:
    db_to_df_spiderTrain = pickle.load(file)

spider_train_pd = pd.read_pickle('/content/drive/MyDrive/spider/SPIDER_Simple_cleaned_final.pkl')
spider_train_pd=spider_train_pd.drop(columns=["query_toks","query_toks_no_value","question_toks","operator"])
spider_train_pd['sql_tags'] = spider_train_pd['query'].apply(get_sql_query_level)

spider_test_pd = pd.read_pickle('/content/drive/MyDrive/spider/SPIDER_Simple_cleaned_final_valid.pkl')
spider_test_pd=spider_test_pd.drop(columns=["query_toks","query_toks_no_value","question_toks","operator"])
spider_test_pd['sql_tags'] = spider_test_pd['query'].apply(get_sql_query_level)

In [ ]:
count=0
for  index, row in spider_train_pd.iterrows():
    print(f"Query: {row['query']}, Question: {row['question']}")
    spider_train_pd["question"]
    count+=1
    if count>30:
      break

In [ ]:
spider_train_pd["query"].head(60)

In [ ]:
# Function to clean the DataFrame
def clean_dataframe(df):
    # Remove quotes from column names
    columns=["question","query"]
    # Remove quotes from each cell
    for col in  columns:
        df[col] = df[col].apply(lambda x: re.sub(r'[\'`"]', '', str(x)) if pd.notnull(x) else x)

    return df
def get_schema(row):
    table_name = row["table_used"]
    return list(db_to_df_proprietary[table_name].columns)

cleaned_df = clean_dataframe(qatch_pickle)
cleaned_df["schema"] = cleaned_df.apply(get_schema, axis=1)
cleaned_df['sql_tags_trunc'] = cleaned_df['sql_tags'].apply(lambda x: x.split('-')[0])## So that we can Having/select Having grouped together and not their subcategories

In [ ]:
pd.options.display.max_colwidth = 100

In [ ]:
print(cleaned_df.loc[[1845]]["table_used"])
print(cleaned_df.loc[[1845]]["query"])
print(cleaned_df.loc[[1845]]["question"])
print(cleaned_df.loc[[1845]]["schema"])
print(cleaned_df.loc[[1845]]["sql_tags"])

1845    breastCancer
Name: table_used, dtype: object
1845    SELECT MAX(estrogenreceptor) FROM breastCancer
Name: query, dtype: object
1845    Find the maximum estrogenreceptor for the table breastCancer
Name: question, dtype: object
1845    [patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...
Name: schema, dtype: object
1845    SIMPLE-AGG-MAX
Name: sql_tags, dtype: object


In [ ]:
print(cleaned_df.loc[[127]]["sql_tags_trunc"])
print(cleaned_df.loc[[127]]["table_used"])
print(cleaned_df.loc[[127]]["query"])
print(cleaned_df.loc[[127]]["question"])

127    SELECT
Name: sql_tags_trunc, dtype: object
127    heartAttack
Name: table_used, dtype: object
127    SELECT restingelectrocardiographicrresults, numberofmajorvvessels FROM heartAttack
Name: query, dtype: object
127    Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack
Name: question, dtype: object


In [ ]:
cleaned_df

,ID,answer_text,db_id,table_used,sql_tags,query,question,domain,operator,answer_coordinates,float_value,schema,sql_tags_trunc
116,116,"[62, male, typicalAngina, 140, 268, false, normal, 2, 2, noHeartAttack, 64, female, asymptomatic...",propiretary4,heartAttack,SELECT-ALL,SELECT * FROM heartAttack,Show all the rows in the table heartAttack,medicine,NaN,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1),...",NaN,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",SELECT
117,117,"[62, 64, 68, 50, 76, 54, 54, 74, 43, 41, 41, 45, 69, 58, 54]",propiretary4,heartAttack,SELECT-ADD-COL,SELECT age FROM heartAttack,Show all age in the table heartAttack,medicine,NaN,"[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0...",NaN,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",SELECT
118,118,"[62, male, 64, female, 68, female, 50, female, 76, male, 54, female, 54, female, 74, male, 43, f...",propiretary4,heartAttack,SELECT-ADD-COL,"SELECT age, sex FROM heartAttack","Show all age, sex in the table heartAttack",medicine,NaN,"[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1), (3, 0), (3, 1), (4, 0), (4, 1), (5, 0), (5, 1),...",NaN,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",SELECT
119,119,"[62, male, typicalAngina, 64, female, asymptomatic, 68, female, nonAnginalPain, 50, female, typi...",propiretary4,heartAttack,SELECT-ADD-COL,"SELECT age, sex, chestpaintype FROM heartAttack","Show all age, sex, chestpaintype in the table heartAttack",medicine,NaN,"[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2), (3, 0), (3, 1), (3, 2),...",NaN,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",SELECT
120,120,"[62, male, typicalAngina, 140, 64, female, asymptomatic, 110, 68, female, nonAnginalPain, 180, 5...",propiretary4,heartAttack,SELECT-ADD-COL,"SELECT age, sex, chestpaintype, restingbloodpressure FROM heartAttack","Show all age, sex, chestpaintype, restingbloodpressure in the table heartAttack",medicine,NaN,"[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3),...",NaN,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",SELECT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,1840,[1],propiretary4,breastCancer,SIMPLE-AGG-MIN,SELECT MIN(numberpositivelymphnodes) FROM breastCancer,Find the minimum numberpositivelymphnodes for the table breastCancer,medicine,NaN,"[(1, 5)]",1.0,"[patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...",SIMPLE
1842,1842,[412],propiretary4,breastCancer,SIMPLE-AGG-MAX,SELECT MAX(progesteronereceptor) FROM breastCancer,Find the maximum progesteronereceptor for the table breastCancer,medicine,NaN,"[(3, 6)]",412.0,"[patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...",SIMPLE
1843,1843,[0],propiretary4,breastCancer,SIMPLE-AGG-MIN,SELECT MIN(progesteronereceptor) FROM breastCancer,Find the minimum progesteronereceptor for the table breastCancer,medicine,NaN,"[(6, 6)]",0.0,"[patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...",SIMPLE
1845,1845,[339],propiretary4,breastCancer,SIMPLE-AGG-MAX,SELECT MAX(estrogenreceptor) FROM breastCancer,Find the maximum estrogenreceptor for the table breastCancer,medicine,NaN,"[(3, 7)]",339.0,"[patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...",SIMPLE


In [ ]:
#@title Legacy code
messages = [{"role": "system",
                     "content": "You are an helpful assistant that assists the user in generating variations of the question."
                                "Given a question and the corresponding SQL version,"
                                " return in JSON format at least 5 possible question variations "
                                "that are coherent with the SQL."
                                "Return the function in a JSON format where the key is the version number"
                                " and the value is the variate question."
                     },

                    {"role": "user", "content":
                        f"Question: What is the average price of the product?\nQuery: SELECT AVG(price) FROM products"
                     },
                    {"role": "system",
                     "content": "{\"1\": \"What is the mean cost of the product?\", "
                                "\"2\": \"Could you tell me the average cost of the product?\", "
                                "\"3\": \"What's the calculated average price of the product?\","
                                " \"4\": \"Can you provide the average price of the product?\""
                                " \"5\": \"I'm interested to know the average price of the product\" }"},

                    {"role": "user", "content":
                        f"Question: {question}\nQuery: {query}"}]

response = openai_client.get_chat_completion(messages)
try:
    response = eval(response)
except Exception as e:
    print(e)
# save to json
# create dataframe
df = pd.DataFrame({
    'schema': " | ".join(schema),
    'response': str(response),
    'question': question,
    'query': query
}, index=[0])

class OpenAICaller:
    def __init__(self, model_name, api_key=None, organization=None):
        self._client = None
        self.api_key = api_key if api_key is not None else os.getenv('OPENAI_API_KEY')
        self.model_name = model_name
        self.organization = organization if organization is not None else os.getenv('OPENAI_API_ORG')

    @property
    def client(self):
        if self._client is None:
            self._client = OpenAI(
                organization=self.organization,
                api_key=self.api_key,
            )
        return self._client

    def get_embedding(self, text):
        return self.client.embeddings.create(input=[text], model=self.model_name).data[0].embedding

    def get_chat_completion(self, messages: list[dict], temperature=None):
        response = self.client.chat.completions.create(model=self.model_name,
                                                       messages=messages,
                                                       temperature=temperature,
                                                       # response_format={"type": "json_object"},
                                                       seed=2024)
        logging.info(f"Num of tokens processed: {response.usage.total_tokens}")
        return response.choices[0].message.content

In [ ]:
import getpass
import os


def SetEnv(name):
    secret = getpass.getpass(f"Enter value of {name}: ")
    os.environ[name] = secret

# # to get var
# mys = os.environ['secret']
# print(mys)

# Usage to set var
SetEnv("OPENAI_API_KEY")
SetEnv("OPENAI_API_ORG")

Enter value of OPENAI_API_KEY: ··········
Enter value of OPENAI_API_ORG: ··········


In [ ]:
#@title OpenAICaller class and some helper fucntions


### good pratcices for embddings from the openai cooks books as the embdedding utils does not work anymore
##3 https://cookbook.openai.com/examples/using_embeddings
import os
import openai
import pandas as pd
import logging
import json

from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI


def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Define the OpenAICaller class
class OpenAICaller:
    def __init__(self, model_name, temperature=None ,api_key=None, organization=None):
        self._client = None
        self.api_key = api_key if api_key is not None else os.getenv('OPENAI_API_KEY')
        self.model_name = model_name
        self.organization = organization if organization is not None else os.getenv('OPENAI_API_ORG')
        self.temperature=temperature

    @property
    def client(self):
        if self._client is None:
            self._client = OpenAI(
                organization=self.organization,
                api_key=self.api_key,
            )
        return self._client

    @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))##not to get rate limits on the api
    def get_embedding(self,text: str, model="text-embedding-3-small") -> list[float]:
        return self.client.embeddings.create(input=[text], model=model).data[0].embedding

    def get_chat_completion(self, messages: list[dict]):
        response = self.client.chat.completions.create(model=self.model_name,
                                                       messages=messages,
                                                       temperature=self.temperature,
                                                       # response_format={"type": "json_object"},
                                                       seed=2024)
        logging.info(f"Num of tokens processed: {response.usage.total_tokens}")
        return response.choices[0].message.content
# Define the function to generate variations
def generate_question_variations(question, query, schema, temperature,model_name="gpt-3.5-turbo"):
    schema_str = " | ".join(schema)

    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant that assists the user in generating variations of the question."
                " Given a question and the corresponding SQL version and schema,"
                " return in JSON format at least 5 possible question variations"
                " that are coherent with the SQL and schema."
                " Return the function in a JSON format where the key is the version number"
                " and the value is the variate question."
            )
        },
        {
            "role": "user",
            "content": f"Schema: {schema_str}\nQuestion: What is the average price of the product?\nQuery: SELECT AVG(price) FROM products"
        },
        {
            "role": "system",
            "content": json.dumps({
                "1": "What is the mean cost of the product?",
                "2": "Could you tell me the average cost of the product?",
                "3": "What's the calculated average price of the product?",
                "4": "Can you provide the average price of the product?",
                "5": "I'm interested to know the average price of the product"
            })
        },
        {
            "role": "user",
            "content": f"Schema: {schema_str}\nQuestion: {question}\nQuery: {query}"
        }
    ]

    # Initialize OpenAICaller and get the response
    openai_caller = OpenAICaller(model_name)
    response = openai_caller.get_chat_completion(messages)

    try:
        response_dict = json.loads(response)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        response_dict = {}

    # Save to JSON and create dataframe
    df = pd.DataFrame({
        'schema': [schema_str],
        'response': [response_dict],
        'question': [question],
        'query': [query]
    })

    return df
# Define function to calculate similarity between original and generated questions
def calculate_similarities(original_question, variations, model_name="text-embedding-3-small"):
    openai_caller = OpenAICaller(model_name)
    original_embedding=openai_caller.get_embedding(original_question,model_name)
    similarities = {}
    for key, variation in variations.items():
        variation_embedding = openai_caller.get_embedding(variation,model_name)
        similarity = cosine_similarity(
            np.array(original_embedding),
            np.array(variation_embedding)
        )
        similarities[key] = similarity

    return similarities


### Tempereature note from openai doc
temperature
number

Optional
Defaults to 0
The sampling temperature, between 0 and 1. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. If set to 0, the model will use log probability to automatically increase the temperature until certain thresholds

### First Table of medicine

In [ ]:
schema = ['age', 'sex', 'chestpaintype', 'restingbloodpressure',
       'cholestoralinmg', 'fastingbloodsugar',
       'restingelectrocardiographicrresults', 'numberofmajorvvessels', 'thall',
       'output']
query = "SELECT * FROM 'heartAttack'"
question = "Show all the rows in the table heartAttack"
temeprature=0
model_name="gpt-3.5-turbo"
df = generate_question_variations(question, query, schema,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

age | sex | chestpaintype | restingbloodpressure | cholestoralinmg | fastingbloodsugar | restingelectrocardiographicrresults | numberofmajorvvessels | thall | output
1: Display all the records in the heartAttack table.
2: List out all the entries in the heartAttack table.
3: Show me the complete data in the heartAttack table.
4: Present all the rows available in the heartAttack table.
5: Could you show me all the information stored in the heartAttack table?
Show all the rows in the table heartAttack
SELECT * FROM 'heartAttack'


In [ ]:
schema = ['age', 'sex', 'chestpaintype', 'restingbloodpressure',
       'cholestoralinmg', 'fastingbloodsugar',
       'restingelectrocardiographicrresults', 'numberofmajorvvessels', 'thall',
       'output']
query = "SELECT * FROM 'heartAttack'"
question = "Show all the rows in the table heartAttack"
temeprature=1
model_name="gpt-3.5-turbo"
df = generate_question_variations(question, query, schema,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

age | sex | chestpaintype | restingbloodpressure | cholestoralinmg | fastingbloodsugar | restingelectrocardiographicrresults | numberofmajorvvessels | thall | output
1: Display all records from the heartAttack table.
2: List all entries available in the heartAttack table.
3: Can you show me the complete data from the heartAttack table?
4: I want to see all the rows from the heartAttack table.
5: Retrieve and show all the information stored in the heartAttack table.
Show all the rows in the table heartAttack
SELECT * FROM 'heartAttack'


In [ ]:
schema = ['age', 'sex', 'chestpaintype', 'restingbloodpressure',
       'cholestoralinmg', 'fastingbloodsugar',
       'restingelectrocardiographicrresults', 'numberofmajorvvessels', 'thall',
       'output']
query = "SELECT * FROM 'heartAttack'"
question = "Show all the rows in the table heartAttack"
temeprature=0.5
model_name="gpt-3.5-turbo"
df = generate_question_variations(question, query, schema,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

age | sex | chestpaintype | restingbloodpressure | cholestoralinmg | fastingbloodsugar | restingelectrocardiographicrresults | numberofmajorvvessels | thall | output
1: Display all records from the heartAttack table.
2: List all entries available in the heartAttack table.
3: Can you show me the complete data from the heartAttack table?
4: I want to see all the rows from the heartAttack table.
5: Retrieve all the rows stored in the heartAttack table.
Show all the rows in the table heartAttack
SELECT * FROM 'heartAttack'


### Second table of medicine

In [ ]:
schema = ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status']
query = "SELECT MAX(estrogenreceptor) FROM breastCancer"
question = "Find the maximum estrogenreceptor for the table breastCancer"
temeprature=0.2
model_name="gpt-4-turbo"
df_select_max_gpt4_turbo = generate_question_variations(question, query, schema,temeprature,model_name)
# Print the output
for index, row in df_select_max_gpt4_turbo.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
1: What is the highest value of estrogen receptor recorded in the breastCancer table?
2: Can you determine the maximum estrogen receptor level in the breastCancer dataset?
3: What's the maximum amount of estrogen receptor found in the breastCancer table?
4: Could you provide the highest estrogen receptor measurement from the breastCancer data?
5: I need to know the maximum estrogen receptor level in the breastCancer table, can you find that?
Find the maximum estrogenreceptor for the table breastCancer
SELECT MAX(estrogenreceptor) FROM breastCancer


In [ ]:
schema = ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status']
query = "SELECT MAX(estrogenreceptor) FROM breastCancer"
question = "Find the maximum estrogenreceptor for the table breastCancer"
temeprature=1
model_name="gpt-3.5-turbo"
df_select_max = generate_question_variations(question, query, schema,temeprature,model_name)
# Print the output
for index, row in df_select_max.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
1: What is the highest value of estrogenreceptor in the breastCancer table?
2: Could you determine the maximum estrogenreceptor value in the breastCancer table?
3: What is the peak estrogenreceptor value in the breastCancer table?
4: Can you find the maximum estrogenreceptor in the breastCancer table?
5: I'd like to know the maximum estrogenreceptor value in the breastCancer table, please.
Find the maximum estrogenreceptor for the table breastCancer
SELECT MAX(estrogenreceptor) FROM breastCancer


In [ ]:
schema = ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status']
query = "SELECT MAX(estrogenreceptor) FROM breastCancer"
question = "Find the maximum estrogenreceptor for the table breastCancer"
temeprature=0
model_name="gpt-3.5-turbo"
df = generate_question_variations(question, query, schema,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
1: What is the highest recorded estrogen receptor level for the breastCancer table?
2: Could you determine the maximum estrogen receptor value in the breastCancer table?
3: I'd like to know the peak estrogen receptor value in the breastCancer table, please.
4: What is the maximum estrogen receptor measurement in the breastCancer table?
5: Please provide the largest estrogen receptor value found in the breastCancer table.
Find the maximum estrogenreceptor for the table breastCancer
SELECT MAX(estrogenreceptor) FROM breastCancer


### Embeddings

| Model       | Usage        |
|-------------|--------------|
| text-embedding-3-small | \$0.02 / 1M tokens |
| text-embedding-3-large | \$0.13 / 1M tokens |
| ada v2      | \$0.10 / 1M tokens |


In [ ]:
variations = df['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": original_question,
        "Variation": variation,
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df


{'1': 0.8132577504065759, '2': 0.8689761233742573, '3': 0.7728410601477239, '4': 0.8561276813156258, '5': 0.8509990764701924}


In [ ]:
variations = df_select_max['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": original_question,
        "Variation": variation,
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df


,Original Question,Variation,Similarity
0,Find the maximum estrogenreceptor for the table breastCancer,What is the highest value of estrogenreceptor in the breastCancer table?,0.845365
1,Find the maximum estrogenreceptor for the table breastCancer,Could you determine the maximum estrogenreceptor value in the breastCancer table?,0.878510
2,Find the maximum estrogenreceptor for the table breastCancer,What is the peak estrogenreceptor value in the breastCancer table?,0.810015
3,Find the maximum estrogenreceptor for the table breastCancer,Can you find the maximum estrogenreceptor in the breastCancer table?,0.920852
4,Find the maximum estrogenreceptor for the table breastCancer,"I'd like to know the maximum estrogenreceptor value in the breastCancer table, please.",0.843058


In [ ]:
variations = df_select_max_gpt4_turbo['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": original_question,
        "Variation": variation,
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df


,Original Question,Variation,Similarity
0,Find the maximum estrogenreceptor for the table breastCancer,What is the highest value of estrogen receptor recorded in the breastCancer table?,0.759241
1,Find the maximum estrogenreceptor for the table breastCancer,Can you determine the maximum estrogen receptor level in the breastCancer dataset?,0.810619
2,Find the maximum estrogenreceptor for the table breastCancer,What's the maximum amount of estrogen receptor found in the breastCancer table?,0.783031
3,Find the maximum estrogenreceptor for the table breastCancer,Could you provide the highest estrogen receptor measurement from the breastCancer data?,0.719364
4,Find the maximum estrogenreceptor for the table breastCancer,"I need to know the maximum estrogen receptor level in the breastCancer table, can you find that?",0.775576


In [ ]:
variations = df_select_max_gpt4_turbo['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations
    # model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": original_question,
        "Variation": variation,
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df


,Original Question,Variation,Similarity
0,Find the maximum estrogenreceptor for the table breastCancer,What is the highest value of estrogen receptor recorded in the breastCancer table?,0.808339
1,Find the maximum estrogenreceptor for the table breastCancer,Can you determine the maximum estrogen receptor level in the breastCancer dataset?,0.817000
2,Find the maximum estrogenreceptor for the table breastCancer,What's the maximum amount of estrogen receptor found in the breastCancer table?,0.848332
3,Find the maximum estrogenreceptor for the table breastCancer,Could you provide the highest estrogen receptor measurement from the breastCancer data?,0.778651
4,Find the maximum estrogenreceptor for the table breastCancer,"I need to know the maximum estrogen receptor level in the breastCancer table, can you find that?",0.857138


### Semi deterministic approach  to contruct the questions based on the strcurted data that qatch uses to generate the questions:
#### Plan:
Go one step at the time:
- first change a bit the english,
- then change a bit also the labels
- do not assume the user knows the perfect ones

In [ ]:
#@title OpenAICaller class and some helper fucntions


### good pratcices for embddings from the openai cooks books as the embdedding utils does not work anymore
##3 https://cookbook.openai.com/examples/using_embeddings
import os
import openai
import pandas as pd
import logging
import json

from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI


def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Define the OpenAICaller class
class OpenAICaller:
    def __init__(self, model_name, temperature=None ,api_key=None, organization=None):
        self._client = None
        self.api_key = api_key if api_key is not None else os.getenv('OPENAI_API_KEY')
        self.model_name = model_name
        self.organization = organization if organization is not None else os.getenv('OPENAI_API_ORG')
        self.temperature=temperature

    @property
    def client(self):
        if self._client is None:
            self._client = OpenAI(
                organization=self.organization,
                api_key=self.api_key,
            )
        return self._client

    @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
    def get_embedding(self,text: str, model="text-embedding-3-small") -> list[float]:
        return self.client.embeddings.create(input=[text], model=model).data[0].embedding

    def get_chat_completion(self, messages: list[dict]):
        response = self.client.chat.completions.create(model=self.model_name,
                                                       messages=messages,
                                                       temperature=self.temperature,
                                                       # response_format={"type": "json_object"},
                                                       seed=2024)
        logging.info(f"Num of tokens processed: {response.usage.total_tokens}")
        return response.choices[0].message.content
# Define the function to generate variations
def generate_question_variations(structured_data,temperature,model_name="gpt-3.5-turbo"):
    schema_str = " | ".join(structured_data['schema'])

    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant that assists the user in generating variations of the question."
                " Given a question, the corresponding SQL version, schema, the table name, the operation and the condition and if some these elements is None ignore it."
                " return in JSON format at least 5 possible question variations"
                " that are coherent with the SQL and the provided metadata."
                " Return the function in a JSON format where the key is the version number"
                " and the value is the variate question."
                "Ensure the questions has some reference to the relevant column names in the schema, the table name ,the operation and the condition "
                "You can use synonyms for the core words and vary the phrasing by using different synonyms and filler words while maintaining the core elements. "
                "Do not explicitly mention SQL clauses like 'GROUP BY' or 'AVG' ... "
                "It is very important that you change the order of the sentence"
                "start with different parts of the sentence each time, and be creative with the phrasing. "
                # "Avoid starting each question with a verb. "
                "Each generated question must be unique and structured differently from the others."
                "The generated questions must be traceable back to the original SQL query."
            )
        },
        {
            "role": "user",
            "content": """Question: What is the average temperature for each city in the weather_data table?
                        \nQuery: SELECT city, AVG(temperature) FROM weather_data GROUP BY city;
                        \schema: department, salary\ntable_name: employees
                        \nOperation: SUM\nCondition: GROUP BY\nType: GROUPBY-AGG-SUM"""
        },
        {
            "role": "system",
            "content": json.dumps({
                "1": "For each city in the weather_data table, what is the average temperature?",
                "2": "The average temperature for each city in the weather_data table is what?",
                "3": "In the weather_data table, what is the mean temperature for every city?",
                "4": "What is the mean temperature per city listed in the weather_data table?",
                "5": "The weather_data table shows what average temperature for each city?"
            })
        },
        {
            "role": "user",
            "content": f"Question: {structured_data['question']}\nQuery: {structured_data['query']}\nschema: {', '.join(structured_data['schema'])}\ntable_name: {structured_data['table_name']}\nOperation: {structured_data['operation']}\nCondition: {structured_data['condition']}"
        }
    ]

    # Initialize OpenAICaller and get the response
    openai_caller = OpenAICaller(model_name)
    response = openai_caller.get_chat_completion(messages)

    try:
        response_dict = json.loads(response)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        response_dict = {}

    # Save to JSON and create dataframe
    df = pd.DataFrame({
        'schema': [schema_str],
        'response': [response_dict],
        'question': [question],
        'query': [query]
    })

    return df
# Define function to calculate similarity between original and generated questions
def calculate_similarities(original_question, variations, model_name="text-embedding-3-large"):
    openai_caller = OpenAICaller(model_name)
    original_embedding=openai_caller.get_embedding(original_question,model_name)
    similarities = {}
    for key, variation in variations.items():
        variation_embedding = openai_caller.get_embedding(variation,model_name)
        similarity = cosine_similarity(
            np.array(original_embedding),
            np.array(variation_embedding)
        )
        similarities[key] = similarity

    return similarities


In [ ]:
structured_input = {
    'table_name': 'breastCancer',
    'schema': ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status'],
    'operation': 'MAX',
    'condition': 'None',
    'query': 'SELECT MAX(estrogenreceptor) FROM breastCancer;',
    'question': 'Find the maximum estrogenreceptor for the table breastCancer'
}
temeprature=0
model_name="gpt-3.5-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
1: What is the highest value of estrogenreceptor in the breastCancer table?
2: The breastCancer table displays the maximum value of estrogenreceptor as what?
3: Obtain the maximum estrogenreceptor value from the breastCancer table.
4: What is the peak estrogenreceptor level recorded in the breastCancer table?
5: The breastCancer table showcases the highest estrogenreceptor value, right?
Find the maximum estrogenreceptor for the table breastCancer
SELECT MAX(estrogenreceptor) FROM breastCancer


In [ ]:
variations = df['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": original_question,
        "Variation": variation,
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original Question,Variation,Similarity
0,Find the maximum estrogenreceptor for the table breastCancer,What is the highest value of estrogenreceptor in the breastCancer table?,0.786828
1,Find the maximum estrogenreceptor for the table breastCancer,The breastCancer table displays the maximum value of estrogenreceptor as what?,0.779837
2,Find the maximum estrogenreceptor for the table breastCancer,Obtain the maximum estrogenreceptor value from the breastCancer table.,0.899279
3,Find the maximum estrogenreceptor for the table breastCancer,What is the peak estrogenreceptor level recorded in the breastCancer table?,0.740059
4,Find the maximum estrogenreceptor for the table breastCancer,"The breastCancer table showcases the highest estrogenreceptor value, right?",0.765707


In [ ]:
structured_input = {
    'table_name': 'breastCancer',
    'schema': ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status'],
    'operation': 'MAX',
    'condition': 'None',
    'query': 'SELECT MAX(estrogenreceptor) FROM breastCancer;',
    'question': 'Find the maximum estrogenreceptor for the table breastCancer'
}
temeprature=0
model_name="gpt-4-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
1: What is the highest value of estrogen receptor in the breastCancer table?
2: Can you determine the maximum estrogen receptor level recorded in the breastCancer table?
3: What's the upper limit of estrogen receptor readings in the breastCancer dataset?
4: In the table breastCancer, what is the peak estrogen receptor value?
5: What maximum estrogen receptor score can be found in the breastCancer database?
Find the maximum estrogenreceptor for the table breastCancer
SELECT MAX(estrogenreceptor) FROM breastCancer


In [ ]:
variations = df['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    # model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": original_question,
        "Variation": variation,
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original Question,Variation,Similarity
0,Find the maximum estrogenreceptor for the table breastCancer,What is the highest value of estrogen receptor in the breastCancer table?,0.768182
1,Find the maximum estrogenreceptor for the table breastCancer,Can you determine the maximum estrogen receptor level recorded in the breastCancer table?,0.799682
2,Find the maximum estrogenreceptor for the table breastCancer,What's the upper limit of estrogen receptor readings in the breastCancer dataset?,0.712241
3,Find the maximum estrogenreceptor for the table breastCancer,"In the table breastCancer, what is the peak estrogen receptor value?",0.767685
4,Find the maximum estrogenreceptor for the table breastCancer,What maximum estrogen receptor score can be found in the breastCancer database?,0.709688


In [ ]:
variations = {
1: "What is the lowest value of estrogen receptor in the breastCancer table?",
2: "Find the average estrogenreceptor for the table breastCancer.",
3: "Count the number of records with a non-zero estrogenreceptor value in the breastCancer table.",
4: "Determine if any record in the breastCancer table has an estrogenreceptor value greater than 100.",
5: "List all estrogenreceptor values from the breastCancer table in descending order."
}
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": original_question,
        "Variation": variation,
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original Question,Variation,Similarity
0,Find the maximum estrogenreceptor for the table breastCancer,What is the lowest value of estrogen receptor in the breastCancer table?,0.715167
1,Find the maximum estrogenreceptor for the table breastCancer,Find the average estrogenreceptor for the table breastCancer.,0.805861
2,Find the maximum estrogenreceptor for the table breastCancer,Count the number of records with a non-zero estrogenreceptor value in the breastCancer table.,0.650419
3,Find the maximum estrogenreceptor for the table breastCancer,Determine if any record in the breastCancer table has an estrogenreceptor value greater than 100.,0.689062
4,Find the maximum estrogenreceptor for the table breastCancer,List all estrogenreceptor values from the breastCancer table in descending order.,0.749651


In [ ]:
#@title Ask the model to provide a minimaltic desciption of the querry using a sbset fo words of the querry for the mendding to work well


import os
import openai
import pandas as pd
import logging
import json
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Define the OpenAICaller class
class OpenAICaller:
    def __init__(self, model_name, temperature=None, api_key=None, organization=None):
        self._client = None
        self.api_key = api_key if api_key is not None else os.getenv('OPENAI_API_KEY')
        self.model_name = model_name
        self.organization = organization if organization is not None else os.getenv('OPENAI_API_ORG')
        self.temperature = temperature

    @property
    def client(self):
        if self._client is None:
            self._client = OpenAI(
                organization=self.organization,
                api_key=self.api_key,
            )
        return self._client

    @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
    def get_embedding(self, text: str, model="text-embedding-3-small") -> list[float]:
        return self.client.embeddings.create(input=[text], model=model).data[0].embedding

    def get_chat_completion(self, messages: list[dict]):
        response = self.client.chat.completions.create(model=self.model_name,
                                                       messages=messages,
                                                       temperature=self.temperature,
                                                       seed=2024)
        logging.info(f"Num of tokens processed: {response.usage.total_tokens}")
        return response.choices[0].message.content

def extract_key_elements(question):
    words = question.split()
    minimal_description = " ".join([word for word in words if word in ["average", "temperature", "city", "weather_data"]])
    return minimal_description

# Define the function to generate variations
def generate_question_variations(structured_data, temperature, model_name="gpt-3.5-turbo"):
    schema_str = " | ".join(structured_data['schema'])
    minimal_description = extract_key_elements(structured_data['question'])

    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant that assists the user in generating variations of the question."
                " Given a question and its corresponding SQL query, schema, table name, operation, and condition (if any),"
                " return at least 5 possible question variations in JSON format."
                " Each item in the JSON output should be a dictionary with the key as the question number and the value as a list containing two elements:"
                " the first element should be the natural language question, and the second element should be a minimalistic description."
                " The minimalistic description should only use words from the original question except for alphanumeric and non-letter characters in words which should be transcribed to English."
                " Ensure the questions reference the relevant columns, table name, operation, and condition, only in plain natural language not alphanumeric and non-letter characters."
                " Use synonyms and vary the phrasing while maintaining core elements."
                " Do not explicitly mention SQL clauses like 'GROUP BY' or 'AVG'."
                " Use English words, not concatenated English words as is the case for some columns with alphanumeric, take care to transcribe them to English."
                " Change the order of the sentence and be creative with the phrasing."
                " Each generated question must be unique and structured differently from the others."
                " The generated questions must be traceable back to the original SQL query."

            )
        },
        {
            "role": "user",
            "content": """Question:Find  the average temperature in each city from the weather_data?
                        \nQuery: SELECT city, AVG(temperature) FROM weather_data GROUP BY city;
                        \nSchema: city, temperature\nTable_name: weather_data\nOperation: AVG\nCondition: GROUP BY"""
        },
        {
            "role": "system",
            "content": json.dumps({
                "original_question": ["Find  the average temperature in each city from the weather_data?", "average temperature city weather data"],
                "1": ["What is the mean temeprature in the diffrent cities in the weather data table?", "mean temperature cities weather data"],
                "2": ["Using the weather_data, What is the usual climate in terms of temperature for each of the urban areas", "usual temperature urban areas weather data"],
                "3": ["What are the typical thermal conditions in each of these cities that you can find in the weather data table?", "typical thermal conditions cities weather data"],
                "4": ["Could you tell me the average degree of warmth in each of the cities based on the weather data table?", "mean temperature cities weather data"],
                "5": ["What is the norm in terms of heat level in each of the cities from the weather data table?", "norm heat level cities weather data"]
            })
        },
        {
            "role": "user",
            "content": f"Question: {structured_data['question']}\nQuery: {structured_data['query']}\nSchema: {', '.join(structured_data['schema'])}\nTable_name: {structured_data['table_name']}\nOperation: {structured_data['operation']}\nCondition: {structured_data['condition']}"
        }
    ]

    # Initialize OpenAICaller and get the response
    openai_caller = OpenAICaller(model_name, temperature)
    response = openai_caller.get_chat_completion(messages)

    try:
        response_dict = json.loads(response)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        response_dict = {}

    # Save to JSON and create dataframe
    df = pd.DataFrame({
        'schema': [schema_str],
        'response': [response_dict],
        'question': [structured_data['question']],
        'query': [structured_data['query']],
        'minimal_description': [minimal_description]
    })

    return df

# Define function to calculate similarity between original and generated questions
def calculate_similarities(original_question, variations, model_name="text-embedding-3-large"):
    openai_caller = OpenAICaller(model_name)
    original_embedding = openai_caller.get_embedding(variations["original_question"][1], model_name)
    similarities = {}
    for key, variation in variations.items():
        variation_embedding = openai_caller.get_embedding(variation[1], model_name)  # Using only the minimal desciption part
        similarity = cosine_similarity(
            np.array(original_embedding),
            np.array(variation_embedding)
        )
        similarities[key] = similarity

    return similarities


In [ ]:
structured_input = {
    'table_name': 'breastCancer',
    'schema': ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status'],
    'operation': 'MAX',
    'condition': 'None',
    'query': 'SELECT MAX(estrogenreceptor) FROM breastCancer;',
    'question': 'Find the maximum estrogenreceptor for the table breastCancer'
}
temeprature=0
model_name="gpt-3.5-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
1: ['What is the highest value of estrogen receptor recorded in the breast cancer dataset?', 'highest value estrogen receptor breast cancer dataset']
2: ['Determine the peak estrogen receptor level among the breast cancer patients in the dataset', 'peak estrogen receptor level breast cancer dataset']
3: ['What is the maximum estrogen receptor value found in the breastCancer table?', 'maximum estrogen receptor value breastCancer table']
4: ['Find the top estrogen receptor measurement in the breastCancer dataset', 'top estrogen receptor measurement breastCancer dataset']
5: ['Identify the highest estrogen receptor value in the breastCancer table', 'highest estrogen receptor value breastCancer table']
Find the maximum estrogenreceptor for the table breastCancer
SELECT MAX(estrogenreceptor) FROM breastCancer;


In [ ]:
variations = df['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": variations["original_question"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

KeyError: 'original_question'

In [ ]:
structured_input = {
    'table_name': 'breastCancer',
    'schema': ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status'],
    'operation': 'MAX',
    'condition': 'None',
    'query': 'SELECT MAX(estrogenreceptor) FROM breastCancer;',
    'question': 'Find the maximum estrogenreceptor for the table breastCancer'
}
temeprature=0.1
model_name="gpt-4-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
original_question: ['Find the maximum estrogen receptor for the table breastCancer?', 'maximum estrogen receptor breast cancer']
1: ['What is the highest value of estrogen receptor recorded in the breastCancer table?', 'highest estrogen receptor breast cancer']
2: ['Can you determine the peak level of estrogen receptor from the breastCancer data?', 'peak estrogen receptor breast cancer']
3: ["What's the maximum measurement of estrogen receptor in the breastCancer database?", 'maximum measurement estrogen receptor breast cancer']
4: ['Identify the top estrogen receptor value in the breastCancer table.', 'top estrogen receptor value breast cancer']
5: ['What is the upper limit of estrogen receptor found in the breastCancer table?', 'upper limit estrogen receptor breast cancer']
Find the maximum estrogenreceptor for the table b

In [ ]:
variations = df['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": variations["original_question"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original Question,Variation,Similarity
0,maximum estrogen receptor breast cancer,maximum estrogen receptor breast cancer,0.999999
1,maximum estrogen receptor breast cancer,highest estrogen receptor breast cancer,0.938388
2,maximum estrogen receptor breast cancer,peak estrogen receptor breast cancer,0.910542
3,maximum estrogen receptor breast cancer,maximum measurement estrogen receptor breast cancer,0.845373
4,maximum estrogen receptor breast cancer,top estrogen receptor value breast cancer,0.865249
5,maximum estrogen receptor breast cancer,upper limit estrogen receptor breast cancer,0.829461


### Go one step at the time:
- then change a bit also the labels
- do not assume the user knows the perfect ones

In [ ]:
#@title change the prompt to hide the labels


import os
import openai
import pandas as pd
import logging
import json
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Define the OpenAICaller class
class OpenAICaller:
    def __init__(self, model_name, temperature=None, api_key=None, organization=None):
        self._client = None
        self.api_key = api_key if api_key is not None else os.getenv('OPENAI_API_KEY')
        self.model_name = model_name
        self.organization = organization if organization is not None else os.getenv('OPENAI_API_ORG')
        self.temperature = temperature

    @property
    def client(self):
        if self._client is None:
            self._client = OpenAI(
                organization=self.organization,
                api_key=self.api_key,
            )
        return self._client

    @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
    def get_embedding(self, text: str, model="text-embedding-3-small") -> list[float]:
        return self.client.embeddings.create(input=[text], model=model).data[0].embedding

    def get_chat_completion(self, messages: list[dict]):
        response = self.client.chat.completions.create(model=self.model_name,
                                                       messages=messages,
                                                       temperature=self.temperature,
                                                       seed=2024)
        logging.info(f"Num of tokens processed: {response.usage.total_tokens}")
        return response.choices[0].message.content

def extract_key_elements(question):
    words = question.split()
    minimal_description = " ".join([word for word in words if word in ["average", "temperature", "city", "weather_data"]])
    return minimal_description

# Define the function to generate variations
def generate_question_variations(structured_data, temperature, model_name="gpt-3.5-turbo"):
    schema_str = " | ".join(structured_data['schema'])
    minimal_description = extract_key_elements(structured_data['question'])

    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant that assists the user in generating variations of the question."
                " Given a question and its corresponding SQL query, schema, table name, operation, and condition (if any),"
                " return at least 5 possible question variations in JSON format."
                " Each item in the JSON output should be a dictionary with the key as the question number and the value as a list containing two elements:"
                " the first element should be the natural language question, and the second element should be a minimalistic description."
                " The minimalistic description should only use words from the original question."
                # " Ensure the questions reference the relevant columns, table name, operation, and condition."
                # " Use synonyms and vary the phrasing while maintaining core elements."
                "Do not assume the user knows the perfect labels/schema"##Added
                " Do not explicitly mention SQL clauses like 'GROUP BY' or 'AVG'."
                " Change the order of the sentence and be creative with the phrasing."
                " Each generated question must be unique and structured differently from the others."
                " The generated questions must be traceable back to the original SQL query."
            )
        },
        {
            "role": "user",
            "content": """Question: What is the average temperature for each city in the weather_data table?
                        \nQuery: SELECT city, AVG(temperature) FROM weather_data GROUP BY city;
                        \nSchema: city, temperature\nTable_name: weather_data\nOperation: AVG\nCondition: GROUP BY"""
        },
        {
            "role": "system",
            "content": json.dumps({
                "original_question": ["What is the average temperature in each city?", "average temperature city weather_data"],
                "1": ["What is the mean temeprature in the diffrent cities in the weather_data table?", "mean temperature cities weather_data"],
                "2": ["Using the weather_data, What is the usual climate in terms of temperature for each of the urban areas", "usual temperature urban areas weather_data"],
                "3": ["What are the typical thermal conditions in each of these cities that you can find in the weather_data table?", "typical thermal conditions cities weather_data"],
                "4": ["Could you tell me the average degree of warmth in each of the cities based on the weather_data table?", "mean temperature cities weather_data"],
                "5": ["What is the norm in terms of heat level in each of the cities from the weather_data table?", "norm heat level cities weather_data"]
            })
        },
        {
            "role": "user",
            "content": f"Question: {structured_data['question']}\nQuery: {structured_data['query']}\nSchema: {', '.join(structured_data['schema'])}\nTable_name: {structured_data['table_name']}\nOperation: {structured_data['operation']}\nCondition: {structured_data['condition']}"
        }
    ]

    # Initialize OpenAICaller and get the response
    openai_caller = OpenAICaller(model_name, temperature)
    response = openai_caller.get_chat_completion(messages)

    try:
        response_dict = json.loads(response)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        response_dict = {}

    # Save to JSON and create dataframe
    df = pd.DataFrame({
        'schema': [schema_str],
        'response': [response_dict],
        'question': [structured_data['question']],
        'query': [structured_data['query']],
        'minimal_description': [minimal_description]
    })

    return df

# Define function to calculate similarity between original and generated questions
def calculate_similarities(original_question, variations, model_name="text-embedding-3-large"):
    openai_caller = OpenAICaller(model_name)
    original_embedding = openai_caller.get_embedding(variations["original_question"][1], model_name)
    similarities = {}
    for key, variation in variations.items():
        variation_embedding = openai_caller.get_embedding(variation[1], model_name)  # Using only the minimal desciption part
        similarity = cosine_similarity(
            np.array(original_embedding),
            np.array(variation_embedding)
        )
        similarities[key] = similarity

    return similarities


In [ ]:
structured_input = {
    'table_name': 'breastCancer',
    'schema': ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status'],
    'operation': 'MAX',
    'condition': 'None',
    'query': 'SELECT MAX(estrogenreceptor) FROM breastCancer;',
    'question': 'Find the maximum estrogenreceptor for the table breastCancer'
}
temeprature=0
model_name="gpt-3.5-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
original_question: ['Find the maximum estrogen receptor for the table breastCancer', 'maximum estrogen receptor breastCancer']
1: ['What is the highest level of estrogen receptor recorded in the breastCancer table?', 'highest level estrogen receptor breastCancer']
2: ['Determine the peak estrogen receptor value in the breastCancer dataset', 'peak estrogen receptor value breastCancer']
3: ['Which patient in the breastCancer table has the highest estrogen receptor value?', 'patient highest estrogen receptor breastCancer']
4: ['Identify the maximum estrogen receptor value among the patients in the breastCancer dataset', 'maximum estrogen receptor patients breastCancer']
5: ['What is the top estrogen receptor measurement found in the breastCancer table?', 'top estrogen receptor measurement breastCancer']
Find the maximum estroge

In [ ]:
variations = df['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": variations["original_question"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original Question,Variation,Similarity
0,maximum estrogen receptor breastCancer,maximum estrogen receptor breastCancer,0.999999
1,maximum estrogen receptor breastCancer,highest level estrogen receptor breastCancer,0.885382
2,maximum estrogen receptor breastCancer,peak estrogen receptor value breastCancer,0.848752
3,maximum estrogen receptor breastCancer,patient highest estrogen receptor breastCancer,0.835628
4,maximum estrogen receptor breastCancer,maximum estrogen receptor patients breastCancer,0.950209
5,maximum estrogen receptor breastCancer,top estrogen receptor measurement breastCancer,0.839296


In [ ]:
structured_input = {
    'table_name': 'breastCancer',
    'schema': ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status'],
    'operation': 'MAX',
    'condition': 'None',
    'query': 'SELECT MAX(estrogenreceptor) FROM breastCancer;',
    'question': 'Find the maximum estrogenreceptor for the table breastCancer'
}
temeprature=0
model_name="gpt-4-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
original_question: ['Find the maximum estrogen receptor level in the breastCancer table.', 'maximum estrogen receptor breastCancer']
1: ['What is the highest level of estrogen receptor recorded in the breastCancer table?', 'highest estrogen receptor breastCancer']
2: ['Can you determine the maximum value of estrogen receptor from the breastCancer table?', 'maximum value estrogen receptor breastCancer']
3: ["What's the peak estrogen receptor level found in the breastCancer table?", 'peak estrogen receptor breastCancer']
4: ['Identify the top estrogen receptor measurement in the breastCancer table.', 'top estrogen receptor measurement breastCancer']
5: ['What is the upper limit of estrogen receptor levels in the breastCancer database?', 'upper limit estrogen receptor breastCancer']
Find the maximum estrogenreceptor for the tab

In [ ]:
variations = df['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": variations["original_question"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original Question,Variation,Similarity
0,maximum estrogen receptor breastCancer,maximum estrogen receptor breastCancer,1.000000
1,maximum estrogen receptor breastCancer,highest estrogen receptor breastCancer,0.947071
2,maximum estrogen receptor breastCancer,maximum value estrogen receptor breastCancer,0.892960
3,maximum estrogen receptor breastCancer,peak estrogen receptor breastCancer,0.912001
4,maximum estrogen receptor breastCancer,top estrogen receptor measurement breastCancer,0.839226
5,maximum estrogen receptor breastCancer,upper limit estrogen receptor breastCancer,0.840433


In [ ]:
structured_input = {
    'table_name': 'breastCancer',
    'schema': ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status'],
    'operation': 'MAX',
    'condition': 'None',
    'query': 'SELECT MAX(estrogenreceptor) FROM breastCancer;',
    'question': 'Find the maximum estrogenreceptor for the table breastCancer'
}
temeprature=0.7
model_name="gpt-4-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
original_question: ['Find the maximum estrogen receptor in the breastCancer table.', 'maximum estrogen receptor breastCancer']
1: ['What is the highest value of estrogen receptor recorded in the breastCancer table?', 'highest estrogen receptor breastCancer']
2: ['Can you determine the maximal estrogen receptor level from the breastCancer table?', 'maximal estrogen receptor breastCancer']
3: ['What is the peak estrogen receptor value in the breastCancer database?', 'peak estrogen receptor breastCancer']
4: ['Identify the top estrogen receptor measurement in the breastCancer table.', 'top estrogen receptor breastCancer']
5: ["What's the highest recorded level of estrogen receptor in the breastCancer table?", 'highest level estrogen receptor breastCancer']
Find the maximum estrogenreceptor for the table breastCancer
SELECT MAX(

In [ ]:
variations = df['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": variations["original_question"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original Question,Variation,Similarity
0,maximum estrogen receptor breastCancer,maximum estrogen receptor breastCancer,1.000000
1,maximum estrogen receptor breastCancer,highest estrogen receptor breastCancer,0.947071
2,maximum estrogen receptor breastCancer,maximal estrogen receptor breastCancer,0.916039
3,maximum estrogen receptor breastCancer,peak estrogen receptor breastCancer,0.912001
4,maximum estrogen receptor breastCancer,top estrogen receptor breastCancer,0.920145
5,maximum estrogen receptor breastCancer,highest level estrogen receptor breastCancer,0.885288


### Text push the ignorance further


In [ ]:
" Assume the user may not be familiar with the exact labels or schema details, and may phrase things in an informal or imprecise way."
" Use synonyms and alternate phrasings to reflect a variety of ways the user might ask the question without knowing the technical terms."


import os
import openai
import pandas as pd
import logging
import json
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Define the OpenAICaller class
class OpenAICaller:
    def __init__(self, model_name, temperature=None, api_key=None, organization=None):
        self._client = None
        self.api_key = api_key if api_key is not None else os.getenv('OPENAI_API_KEY')
        self.model_name = model_name
        self.organization = organization if organization is not None else os.getenv('OPENAI_API_ORG')
        self.temperature = temperature

    @property
    def client(self):
        if self._client is None:
            self._client = OpenAI(
                organization=self.organization,
                api_key=self.api_key,
            )
        return self._client

    @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
    def get_embedding(self, text: str, model="text-embedding-3-small") -> list[float]:
        return self.client.embeddings.create(input=[text], model=model).data[0].embedding

    def get_chat_completion(self, messages: list[dict]):
        response = self.client.chat.completions.create(model=self.model_name,
                                                       messages=messages,
                                                       temperature=self.temperature,
                                                       seed=2024)
        logging.info(f"Num of tokens processed: {response.usage.total_tokens}")
        return response.choices[0].message.content

def extract_key_elements(question):
    words = question.split()
    minimal_description = " ".join([word for word in words if word in ["average", "temperature", "city", "weather_data"]])
    return minimal_description

# Define the function to generate variations
def generate_question_variations(structured_data, temperature, model_name="gpt-3.5-turbo"):
    schema_str = " | ".join(structured_data['schema'])
    minimal_description = extract_key_elements(structured_data['question'])

    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant that assists the user in generating variations of the question."
                " Given a question and its corresponding SQL query, schema, table name, operation, and condition (if any),"
                " return at least 5 possible question variations in JSON format."
                "Do not use the exact words int the query such as columns or whatver reformulate them if they are stuck with each other sperate them it has to be natiral informal language where we simulate that we don;t really know the lables but we have a vague idea just like the way a human would formulate the quetions without refenrencing the exact same lables but something close to them"
                " Each item in the JSON output should be a dictionary with the key as the question number and the value as a list containing two elements:"
                " the first element should be the natural language question, and the second element should be a minimalistic description"
                "The minmalitic desciption  should follow strictly the following template: operations(there could be many) target columns(there could be many) target table"
                " The minimalistic description should only use words from the original question"
                "The minimalistic description shouldn't contain filler words because it is used in embdedding cosine similarity so a single word can skew the similarity be mnindful"
                " Assume the user may not be familiar with the exact labels or schema details, and may phrase things in an informal or imprecise way."## added
                " Use synonyms and alternate phrasings to reflect a variety of ways the user might ask the question without knowing the technical terms."#added
                " Do not explicitly mention SQL clauses like 'GROUP BY' or 'AVG'."
                " Change the order of the sentence and be creative with the phrasing."
                " Each generated question must be unique and structured differently from the others."
                # " The generated questions must be traceable back to the original SQL query."
            )
        },
        {
            "role": "user",
            "content": """Question: Find the average temperature for each city in the weather_data table?
                        \nQuery: SELECT city, AVG(temperature) FROM weather_data GROUP BY city;
                        \nSchema: city, temperature\nTable_name: weather_data\nOperation: AVG\nCondition: GROUP BY"""
        },
        {
            "role": "system",
            "content": json.dumps({
                "original_question": ["Find the average temperature for each city in the weather data table?", "average temperature city weather data"],
                "1": ["What is the mean temeprature in the diffrent cities in the weather data table?", "mean temperature cities weather data"],
                "2": ["Using the weather_data, What is the usual climate in terms of temperature for each of the urban areas", "usual temperature urban areas weather data"],
                "3": ["What are the typical thermal conditions in each of these cities that you can find in the weather data table?", "typical thermal conditions cities weather data"],
                "4": ["Could you tell me the average degree of warmth in each of the cities based on the weather data table?", "mean temperature cities weather data"],
                "5": ["What is the norm in terms of heat level in each of the cities from the weather data table?", "norm heat level cities weather data"]
            })
        },
        {
            "role": "user",
            "content": f"Question: {structured_data['question']}\nQuery: {structured_data['query']}\nSchema: {', '.join(structured_data['schema'])}\nTable_name: {structured_data['table_name']}\nOperation: {structured_data['operation']}\nCondition: {structured_data['condition']}"
        }
    ]

    # Initialize OpenAICaller and get the response
    openai_caller = OpenAICaller(model_name, temperature)
    response = openai_caller.get_chat_completion(messages)

    try:
        response_dict = json.loads(response)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        response_dict = {}

    # Save to JSON and create dataframe
    df = pd.DataFrame({
        'schema': [schema_str],
        'response': [response_dict],
        'question': [structured_data['question']],
        'query': [structured_data['query']],
        'minimal_description': [minimal_description]
    })

    return df

# Define function to calculate similarity between original and generated questions
def calculate_similarities(original_question, variations, model_name="text-embedding-3-large"):
    openai_caller = OpenAICaller(model_name)
    original_embedding = openai_caller.get_embedding(variations["original_question"][1], model_name)
    similarities = {}
    for key, variation in variations.items():
        variation_embedding = openai_caller.get_embedding(variation[1], model_name)  # Using only the minimal desciption part
        similarity = cosine_similarity(
            np.array(original_embedding),
            np.array(variation_embedding)
        )
        similarities[key] = similarity

    return similarities


In [ ]:
structured_input = {
    'table_name': 'breastCancer',
    'schema': ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status'],
    'operation': 'MAX',
    'condition': 'None',
    'query': 'SELECT MAX(estrogenreceptor) FROM breastCancer;',
    'question': 'Find the maximum estrogenreceptor for the table breastCancer'
}
temeprature=0.6
model_name="gpt-3.5-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
original_question: ['Find the maximum estrogenreceptor for the table breastCancer', 'maximum estrogenreceptor breastCancer']
1: ['What is the highest level of estrogenreceptor found in the breastCancer table?', 'highest level estrogenreceptor breastCancer']
2: ['Which patient in the breastCancer table has the most estrogenreceptor?', 'patient most estrogenreceptor breastCancer']
3: ['What is the peak value of estrogenreceptor among the patients in the breastCancer dataset?', 'peak value estrogenreceptor patients breastCancer']
4: ['Who has the maximum estrogenreceptor among the patients in the breastCancer data?', 'maximum estrogenreceptor patients breastCancer']
5: ['What is the top estrogenreceptor value recorded in the breastCancer records?', 'top estrogenreceptor value breastCancer']
Find the maximum estrogenreceptor for

In [ ]:
variations = df['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": variations["original_question"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original Question,Variation,Similarity
0,maximum estrogenreceptor breastCancer,maximum estrogenreceptor breastCancer,0.999999
1,maximum estrogenreceptor breastCancer,highest estrogenreceptor value breastCancer,0.899722
2,maximum estrogenreceptor breastCancer,top estrogenreceptor level breastCancer,0.888256
3,maximum estrogenreceptor breastCancer,patient maximum estrogenreceptor breastCancer,0.853417
4,maximum estrogenreceptor breastCancer,peak estrogenreceptor value patients breastCancer,0.824407
5,maximum estrogenreceptor breastCancer,maximal estrogenreceptor content breastCancer,0.876487


In [ ]:
structured_input = {
    'table_name': 'breastCancer',
    'schema': ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status'],
    'operation': 'MAX',
    'condition': 'None',
    'query': 'SELECT MAX(estrogenreceptor) FROM breastCancer;',
    'question': 'Find the maximum estrogenreceptor for the table breastCancer'
}
temeprature=0.6
model_name="gpt-4-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
original_question: ['Find the maximum estrogenreceptor for the table breastCancer', 'maximum estrogenreceptor breastCancer']
1: ['What is the highest level of estrogen receptor recorded in the breastCancer table?', 'highest estrogen receptor breastCancer']
2: ['Can you determine the peak estrogen receptor value from the breastCancer data?', 'peak estrogen receptor breastCancer']
3: ["What's the top measurement of estrogen receptor found in the breastCancer table?", 'top estrogen receptor breastCancer']
4: ['Could you tell me the maximum value of estrogen receptor present in the breastCancer records?', 'maximum estrogen receptor breastCancer']
5: ['What is the greatest estrogen receptor value registered in the breastCancer table?', 'greatest estrogen receptor breastCancer']
Find the maximum estrogenreceptor for the table brea

In [ ]:
variations = df['response'].iloc[0]
original_question="Find the maximum estrogenreceptor for the table breastCancer"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": variations["original_question"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original Question,Variation,Similarity
0,maximum estrogenreceptor breastCancer,maximum estrogenreceptor breastCancer,1.000000
1,maximum estrogenreceptor breastCancer,highest estrogen receptor breastCancer,0.924438
2,maximum estrogenreceptor breastCancer,peak estrogen receptor breastCancer,0.897117
3,maximum estrogenreceptor breastCancer,top estrogen receptor breastCancer,0.908207
4,maximum estrogenreceptor breastCancer,maximum estrogen receptor breastCancer,0.971832
5,maximum estrogenreceptor breastCancer,greatest estrogen receptor breastCancer,0.875339


In [ ]:
structured_input = {
    'table_name': 'heartAttack',
    'schema': ['age', 'sex', 'chestpaintype', 'restingbloodpressure',
       'cholestoralinmg', 'fastingbloodsugar',
       'restingelectrocardiographicrresults', 'numberofmajorvvessels', 'thall',
       'output'],
    'operation': 'SELECT',
    'condition': 'None',
    'query': 'SELECT restingelectrocardiographicrresults, numberofmajorvvessels FROM heartAttack',
    'question': 'Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack'
}
temeprature=0.6
model_name="gpt-4-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

age | sex | chestpaintype | restingbloodpressure | cholestoralinmg | fastingbloodsugar | restingelectrocardiographicrresults | numberofmajorvvessels | thall | output
original_question: ['Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack', 'restingelectrocardiographicrresults numberofmajorvvessels heartAttack']
1: ['Can you display the cardiac electrical activity results and the count of main blood vessels from the heartAttack table?', 'cardiac electrical activity count main blood vessels heartAttack']
2: ['Could you list the ECG results and the total number of significant arteries reported in the heartAttack table?', 'ECG results total significant arteries heartAttack']
3: ['What are the details of heart electrical monitoring and major vessel counts in the heartAttack table?', 'heart electrical monitoring major vessel counts heartAttack']
4: ['Please provide the electrocardiogram findings and major artery counts from the heartAttack table.', '

In [ ]:
variations = df['response'].iloc[0]
original_question="Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": variations["original_question"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original Question,Variation,Similarity
0,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,0.999999
1,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,cardiac electrical activity count main blood vessels heartAttack,0.663911
2,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,ECG results total significant arteries heartAttack,0.741724
3,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,heart electrical monitoring major vessel counts heartAttack,0.724410
4,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,electrocardiogram findings major artery counts heartAttack,0.722594
5,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,heart electrical test primary vessels heartAttack,0.661057


In [ ]:
structured_input = {
    'table_name': 'heartAttack',
    'schema': ['age', 'sex', 'chestpaintype', 'restingbloodpressure',
       'cholestoralinmg', 'fastingbloodsugar',
       'restingelectrocardiographicrresults', 'numberofmajorvvessels', 'thall',
       'output'],
    'operation': 'SELECT',
    'condition': 'None',
    'query': 'SELECT restingelectrocardiographicrresults, numberofmajorvvessels FROM heartAttack',
    'question': 'Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack'
}
temeprature=0
model_name="gpt-4-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

age | sex | chestpaintype | restingbloodpressure | cholestoralinmg | fastingbloodsugar | restingelectrocardiographicrresults | numberofmajorvvessels | thall | output
original_question: ['Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack', 'SELECT restingelectrocardiographicrresults numberofmajorvvessels heartAttack']
1: ['Can you display the ECG results and the count of major vessels from the heartAttack table?', 'ECG results count major vessels heartAttack']
2: ['I need to see the electrocardiogram findings and the number of significant vessels in the heartAttack table, can you pull that up?', 'electrocardiogram findings number significant vessels heartAttack']
3: ["Could you list the results of the heart's electrical activity and the major vessel counts from the heartAttack table?", "heart's electrical activity major vessel counts heartAttack"]
4: ['Please provide the details on cardiac electrical results and major blood vessel counts from t

In [ ]:
variations = df['response'].iloc[0]
original_question="Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": variations["original_question"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original Question,Variation,Similarity
0,SELECT restingelectrocardiographicrresults numberofmajorvvessels heartAttack,SELECT restingelectrocardiographicrresults numberofmajorvvessels heartAttack,1.000000
1,SELECT restingelectrocardiographicrresults numberofmajorvvessels heartAttack,ECG results count major vessels heartAttack,0.777185
2,SELECT restingelectrocardiographicrresults numberofmajorvvessels heartAttack,electrocardiogram findings number significant vessels heartAttack,0.744524
3,SELECT restingelectrocardiographicrresults numberofmajorvvessels heartAttack,heart's electrical activity major vessel counts heartAttack,0.723791
4,SELECT restingelectrocardiographicrresults numberofmajorvvessels heartAttack,cardiac electrical results major blood vessel counts heartAttack,0.733954
5,SELECT restingelectrocardiographicrresults numberofmajorvvessels heartAttack,readings resting ECG number major arteries heartAttack,0.690208


### Enhanced version with some speficics to the prompt

In [ ]:
" Assume the user may not be familiar with the exact labels or schema details, and may phrase things in an informal or imprecise way."
" Use synonyms and alternate phrasings to reflect a variety of ways the user might ask the question without knowing the technical terms."


import os
import openai
import pandas as pd
import logging
import json
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Define the OpenAICaller class
class OpenAICaller:
    def __init__(self, model_name, temperature=None, api_key=None, organization=None):
        self._client = None
        self.api_key = api_key if api_key is not None else os.getenv('OPENAI_API_KEY')
        self.model_name = model_name
        self.organization = organization if organization is not None else os.getenv('OPENAI_API_ORG')
        self.temperature = temperature

    @property
    def client(self):
        if self._client is None:
            self._client = OpenAI(
                organization=self.organization,
                api_key=self.api_key,
            )
        return self._client

    @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
    def get_embedding(self, text: str, model="text-embedding-3-small") -> list[float]:
        return self.client.embeddings.create(input=[text], model=model).data[0].embedding

    def get_chat_completion(self, messages: list[dict]):
        response = self.client.chat.completions.create(model=self.model_name,
                                                       messages=messages,
                                                       temperature=self.temperature,
                                                       seed=2024)
        logging.info(f"Num of tokens processed: {response.usage.total_tokens}")
        return response.choices[0].message.content

def extract_key_elements(question):
    words = question.split()
    minimal_description = " ".join([word for word in words if word in ["average", "temperature", "city", "weather_data"]])
    return minimal_description

# Define the function to generate variations
def generate_question_variations(structured_data, temperature, model_name="gpt-3.5-turbo"):
    schema_str = " | ".join(structured_data['schema'])
    minimal_description = extract_key_elements(structured_data['question'])

    messages = [
        {
            "role": "system",
            "content": (

        "You are a helpful assistant that assists the user in generating variations of the question."
        " Given a question and its corresponding SQL query, schema, table name, operation, and condition (if any),"
        " return at least 5 possible question variations in JSON format."
        " Do not use the exact words in the query such as columns or other specific terms; separate them if they are stuck together. It has to be natural, informal language where we simulate that we don't really know the labels but have a vague idea, just like the way a human would formulate the questions without referencing the exact same labels but something close to them."
        " Each item in the JSON output should be a dictionary with the key as the question number and the value as a list containing two elements:"
        " the first element should be the natural language question, and the second element should be a minimalistic description."
        " The minimalistic description should follow strictly the following template: operations (there could be many), target columns (there could be many), target table."
        " The minimalistic description should only use words from the original question."
        " The minimalistic description shouldn't contain filler words because it is used in embedding cosine similarity, so a single word can skew the similarity; be mindful."
        " The minimalistic description and the generated natural language questions should only use words from the original question except for alphanumeric and non-letter characters in words which should be transcribed to English."
        " Assume the user may not be familiar with the exact labels or schema details, and may phrase things in an informal or imprecise way."
        " Use synonyms and alternate phrasings to reflect a variety of ways the user might ask the question without knowing the technical terms."
        " Do not explicitly mention SQL clauses like 'GROUP BY' or 'AVG'."
        " Change the order of the sentence and be creative with the phrasing."
        " Each generated question must be unique and structured differently from the others."

                # " The generated questions must be traceable back to the original SQL query."
            )
        },
        {
            "role": "user",
            "content": """Question: Find the average temperature for each city in the weather_data table?
                        \nQuery: SELECT city, AVG(temperature) FROM weather_data GROUP BY city;
                        \nSchema: city, temperature\nTable_name: weather_data\nOperation: AVG\nCondition: GROUP BY"""
        },
        {
            "role": "system",
            "content": json.dumps({
                "original_question": ["Find the average temperature for each city in the weather data table?", "average temperature city weather data"],
                "1": ["What is the mean temeprature in the diffrent cities in the weather data table?", "mean temperature cities weather data"],
                "2": ["Using the weather_data, What is the usual climate in terms of temperature for each of the urban areas", "usual temperature urban areas weather data"],
                "3": ["What are the typical thermal conditions in each of these cities that you can find in the weather data table?", "typical thermal conditions cities weather data"],
                "4": ["Could you tell me the average degree of warmth in each of the cities based on the weather data table?", "mean temperature cities weather data"],
                "5": ["What is the norm in terms of heat level in each of the cities from the weather data table?", "norm heat level cities weather data"]
            })
        },
        {
            "role": "user",
            "content": f"Question: {structured_data['question']}\nQuery: {structured_data['query']}\nSchema: {', '.join(structured_data['schema'])}\nTable_name: {structured_data['table_name']}\nOperation: {structured_data['operation']}\nCondition: {structured_data['condition']}"
        }
    ]

    # Initialize OpenAICaller and get the response
    openai_caller = OpenAICaller(model_name, temperature)
    response = openai_caller.get_chat_completion(messages)

    try:
        response_dict = json.loads(response)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        response_dict = {}

    # Save to JSON and create dataframe
    df = pd.DataFrame({
        'schema': [schema_str],
        'response': [response_dict],
        'question': [structured_data['question']],
        'query': [structured_data['query']],
        'minimal_description': [minimal_description]
    })

    return df

# Define function to calculate similarity between original and generated questions
def calculate_similarities(original_question, variations, model_name="text-embedding-3-large"):
    openai_caller = OpenAICaller(model_name)
    original_embedding = openai_caller.get_embedding(variations["original_question"][1], model_name)
    similarities = {}
    for key, variation in variations.items():
        variation_embedding = openai_caller.get_embedding(variation[1], model_name)  # Using only the minimal desciption part
        similarity = cosine_similarity(
            np.array(original_embedding),
            np.array(variation_embedding)
        )
        similarities[key] = similarity

    return similarities


In [ ]:
structured_input = {
    'table_name': 'heartAttack',
    'schema': ['age', 'sex', 'chestpaintype', 'restingbloodpressure',
       'cholestoralinmg', 'fastingbloodsugar',
       'restingelectrocardiographicrresults', 'numberofmajorvvessels', 'thall',
       'output'],
    'operation': 'SELECT',
    'condition': 'None',
    'query': 'SELECT restingelectrocardiographicrresults, numberofmajorvvessels FROM heartAttack',
    'question': 'Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack'
}
temeprature=0
model_name="gpt-3.5-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

age | sex | chestpaintype | restingbloodpressure | cholestoralinmg | fastingbloodsugar | restingelectrocardiographicrresults | numberofmajorvvessels | thall | output
original_question: ['Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack']
1: ['What are the results of the resting ECG and the count of major vessels in the heartAttack table?', 'results resting ECG count major vessels heartAttack']
2: ['Can you display the resting ECG outcomes and the quantity of significant arteries in the heartAttack dataset?', 'display resting ECG outcomes quantity significant arteries heartAttack']
3: ['List all the resting ECG readings and the number of major blood vessels in the heartAttack data', 'list resting ECG readings number major blood vessels heartAttack']
4: ['What are the resting ECG results and the total major vessels in the heartAttack table?', 'resting ECG results total major vessels heartAttack']
5: ['Display the resting ECG findings and the nu

In [ ]:
variations = df['response'].iloc[0]
original_question="Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": variations["original_question"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

IndexError: list index out of range

In [ ]:
structured_input = {
    'table_name': 'heartAttack',
    'schema': ['age', 'sex', 'chestpaintype', 'restingbloodpressure',
       'cholestoralinmg', 'fastingbloodsugar',
       'restingelectrocardiographicrresults', 'numberofmajorvvessels', 'thall',
       'output'],
    'operation': 'SELECT',
    'condition': 'None',
    'query': 'SELECT restingelectrocardiographicrresults, numberofmajorvvessels FROM heartAttack',
    'question': 'Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack'
}
temeprature=0
model_name="gpt-4-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["question"])
    print(row["query"])

age | sex | chestpaintype | restingbloodpressure | cholestoralinmg | fastingbloodsugar | restingelectrocardiographicrresults | numberofmajorvvessels | thall | output
original_question: ['Show all resting electrocardiographic results, number of major vessels in the table heart Attack', 'resting electrocardiographic results number of major vessels heart Attack']
1: ['Can you display the ECG results and the count of major blood vessels from the heart Attack table?', 'ECG results count major blood vessels heart Attack']
2: ["What are the details on heart's electrical activity and major vessel count in the heart Attack table?", 'heart electrical activity major vessel count heart Attack']
3: ['Could you list the electrocardiogram findings and the number of significant vessels from the heart Attack table?', 'electrocardiogram findings number significant vessels heart Attack']
4: ['Please provide the data on cardiac electrical results and vessel count from the heart Attack table.', 'cardiac el

In [ ]:
variations = df['response'].iloc[0]
original_question="Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack"
similarities = calculate_similarities(
    original_question,
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original Question": variations["original_question"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original Question,Variation,Similarity
0,resting electrocardiographic results number of major vessels heart Attack,resting electrocardiographic results number of major vessels heart Attack,0.999999
1,resting electrocardiographic results number of major vessels heart Attack,ECG results count major blood vessels heart Attack,0.853935
2,resting electrocardiographic results number of major vessels heart Attack,heart electrical activity major vessel count heart Attack,0.800874
3,resting electrocardiographic results number of major vessels heart Attack,electrocardiogram findings number significant vessels heart Attack,0.819329
4,resting electrocardiographic results number of major vessels heart Attack,cardiac electrical results vessel count heart Attack,0.787722
5,resting electrocardiographic results number of major vessels heart Attack,ECG outcomes major vessels heart Attack,0.797634


### Final

In [ ]:
" Assume the user may not be familiar with the exact labels or schema details, and may phrase things in an informal or imprecise way."
" Use synonyms and alternate phrasings to reflect a variety of ways the user might ask the question without knowing the technical terms."


import os
import openai
import pandas as pd
import logging
import json
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Define the OpenAICaller class
class OpenAICaller:
    def __init__(self, model_name, temperature=None, api_key=None, organization=None):
        self._client = None
        self.api_key = api_key if api_key is not None else os.getenv('OPENAI_API_KEY')
        self.model_name = model_name
        self.organization = organization if organization is not None else os.getenv('OPENAI_API_ORG')
        self.temperature = temperature

    @property
    def client(self):
        if self._client is None:
            self._client = OpenAI(
                organization=self.organization,
                api_key=self.api_key,
            )
        return self._client

    @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
    def get_embedding(self, text: str, model="text-embedding-3-small") -> list[float]:
        return self.client.embeddings.create(input=[text], model=model).data[0].embedding

    def get_chat_completion(self, messages: list[dict]):
        response = self.client.chat.completions.create(model=self.model_name,
                                                       messages=messages,
                                                       temperature=self.temperature,
                                                       seed=2024)
        logging.info(f"Num of tokens processed: {response.usage.total_tokens}")
        return response.choices[0].message.content

# def extract_key_elements(question):
#     words = question.split()
#     minimal_description = " ".join([word for word in words if word in ["average", "temperature", "city", "weather_data"]])
#     return minimal_description

# Define the function to generate variations
def generate_question_variations(structured_data, temperature, model_name="gpt-3.5-turbo"):
    schema_str = " | ".join(structured_data['schema'])
    # minimal_description = extract_key_elements(structured_data['question'])

    messages = [
        {
            "role": "system",
            "content": (

        "You are a helpful assistant that assists the user in generating variations of the question."
        " Given a  SQL query, schema, table name,  and a SQL Tag containing operation, and condition (if any),"
        " return at least 5 possible question variations in natural language of the sql query in JSON format."
        " Do not use the exact words in the query such as columns or other specific terms; separate them if they are stuck together. It has to be natural, informal language where we simulate that we don't really know the labels but have a vague idea, just like the way a human would formulate the questions without referencing the exact same labels but something close to them."
        " Each item in the JSON output should be a dictionary with the key as the question number and the value as a list containing two elements:"
        " the first element should be the natural language question, and the second element should be a minimalistic description."
        " The minimalistic description should follow strictly the following template: operations (there could be many), target columns (there could be many), target table."
        " The minimalistic description should only use words from the original question."
        " The minimalistic description shouldn't contain filler words because it is used in embedding cosine similarity, so a single word can skew the similarity; be mindful."
        " The minimalistic description and the generated natural language questions should only use words from the original question except for alphanumeric and non-letter characters in words which should be transcribed to English."
        " Assume the user may not be familiar with the exact labels or schema details, and may phrase things in an informal or imprecise way."
        " Use synonyms and alternate phrasings to reflect a variety of ways the user might ask the question without knowing the technical terms."
        " Do not explicitly mention SQL clauses like 'GROUP BY' or 'AVG'."
        " Change the order of the sentence and be creative with the phrasing."
        " Each generated question must be unique and structured differently from the others."
        # " The generated questions must be traceable back to the original SQL query."##added
            )
        },
        {
            "role": "user",
            "content": """\nQuery: SELECT AVG(temperature) FROM weather_data GROUP BY city;
                        \nSchema: city, temperature\nTable_name: weather_data\nSQL-TAG: GROUPBY-AVG"""
        },
        {
            "role": "system",
            "content": json.dumps({
                "original_query": ["SELECT AVG(temperature) FROM weather_data GROUP BY city;", "average temperature group by city weather data"],
                "1": ["What is the mean temeprature in the diffrent cities in the weather data table?", "mean temperature in cities weather data"],
                "2": ["Using the weather_data, What is the usual climate in terms of temperature for each of the urban areas", "usual temperature for each of urban areas weather data"],
                "3": ["What are the typical thermal conditions in each of these cities that you can find in the weather data table?", "typical thermal conditions in each of cities weather data"],
                "4": ["Could you tell me the average degree of warmth in each of the cities based on the weather data table?", "mean temperature in each of cities weather data"],
                "5": ["What is the norm in terms of heat level in each of the cities from the weather data table?", "norm heat level in each of cities weather data"]
            })
        },
        {
            "role": "user",
            "content": f"Query: {structured_data['query']}\nSchema: {', '.join(structured_data['schema'])}\nTable_name: {structured_data['table_name']}\SQL-TAG: {structured_data['SQL-TAG']}"
        }
    ]

    # Initialize OpenAICaller and get the response
    openai_caller = OpenAICaller(model_name, temperature)
    response = openai_caller.get_chat_completion(messages)

    try:
        response_dict = json.loads(response)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        response_dict = {}

    # Save to JSON and create dataframe
    df = pd.DataFrame({
        'schema': [schema_str],
        'response': [response_dict],
        'query': [structured_data['query']],
        # 'minimal_description': [minimal_description]
    })

    return df

# Define function to calculate similarity between original and generated questions
def calculate_similarities(variations, model_name="text-embedding-3-large"):
    openai_caller = OpenAICaller(model_name)
    original_embedding = openai_caller.get_embedding(variations["original_query"][1], model_name)
    similarities = {}
    for key, variation in variations.items():
        variation_embedding = openai_caller.get_embedding(variation[1], model_name)  # Using only the minimal desciption part
        similarity = cosine_similarity(
            np.array(original_embedding),
            np.array(variation_embedding)
        )
        similarities[key] = similarity

    return similarities


In [ ]:
structured_input = {
    'table_name': 'heartAttack',
    'schema': ['age', 'sex', 'chestpaintype', 'restingbloodpressure',
       'cholestoralinmg', 'fastingbloodsugar',
       'restingelectrocardiographicrresults', 'numberofmajorvvessels', 'thall',
       'output'],
    # 'operation': 'SELECT',
    # 'condition': 'None',
    'SQL-TAG': 'SELECT-RANDOM-COL',
    'query': 'SELECT restingelectrocardiographicrresults, numberofmajorvvessels FROM heartAttack',
    # 'question': 'Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack'
}
temeprature=0
model_name="gpt-3.5-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    # print(row["question"])
    print(row["query"])

age | sex | chestpaintype | restingbloodpressure | cholestoralinmg | fastingbloodsugar | restingelectrocardiographicrresults | numberofmajorvvessels | thall | output
original_query: ['SELECT restingelectrocardiographicrresults, numberofmajorvvessels FROM heartAttack', 'restingelectrocardiographicrresults numberofmajorvvessels heartAttack']
1: ['What are the results of the electrocardiogram at rest and the count of major blood vessels in the heartAttack table?', 'results of electrocardiogram at rest count of major blood vessels heartAttack']
2: ['In the heartAttack table, what can you tell me about the resting electrocardiogram outcomes and the number of significant blood vessels?', 'resting electrocardiogram outcomes number of significant blood vessels heartAttack']
3: ['Could you provide information on the resting heart test results and the quantity of major blood vessels in the heartAttack table?', 'resting heart test results quantity of major blood vessels heartAttack']
4: ['What de

In [ ]:
variations = df['response'].iloc[0]
original_query="SELECT restingelectrocardiographicrresults"
similarities = calculate_similarities(
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original query": variations["original_query"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original query,Variation,Similarity
0,select restingelectrocardiographicresults numberofmajorvessels heartAttack,select restingelectrocardiographicresults numberofmajorvessels heartAttack,1.000000
1,select restingelectrocardiographicresults numberofmajorvessels heartAttack,ECG results count of major blood vessels heartAttack,0.796426
2,select restingelectrocardiographicresults numberofmajorvessels heartAttack,electrocardiographic findings major vessel numbers heartAttack,0.798035
3,select restingelectrocardiographicresults numberofmajorvessels heartAttack,heart's electrical activity number of significant vessels heartAttack,0.762619
4,select restingelectrocardiographicresults numberofmajorvessels heartAttack,resting ECG major vessels noted each case heartAttack,0.759308
5,select restingelectrocardiographicresults numberofmajorvessels heartAttack,electrocardiogram results total major vessels heartAttack,0.786915


In [ ]:
structured_input = {
    'table_name': 'heartAttack',
    'schema': ['age', 'sex', 'chestpaintype', 'restingbloodpressure',
       'cholestoralinmg', 'fastingbloodsugar',
       'restingelectrocardiographicrresults', 'numberofmajorvvessels', 'thall',
       'output'],
    # 'operation': 'SELECT',
    # 'condition': 'None',
    'SQL-TAG': 'SELECT-RANDOM-COL',
    'query': 'SELECT restingelectrocardiographicrresults, numberofmajorvvessels FROM heartAttack',
    # 'question': 'Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack'
}
temeprature=0
model_name="gpt-4-turbo"
df = generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    # print(row["question"])
    print(row["query"])

age | sex | chestpaintype | restingbloodpressure | cholestoralinmg | fastingbloodsugar | restingelectrocardiographicrresults | numberofmajorvvessels | thall | output
original_query: ['SELECT restingelectrocardiographicresults, numberofmajorvessels FROM heartAttack;', 'select restingelectrocardiographicresults numberofmajorvessels heartAttack']
1: ['Can you show me the ECG results and the count of major blood vessels from the heart attack records?', 'ECG results count of major blood vessels heartAttack']
2: ['What are the details regarding the electrocardiographic findings and major vessel numbers in the heart attack data?', 'electrocardiographic findings major vessel numbers heartAttack']
3: ["Could you provide the data on heart's electrical activity and the number of significant vessels from the heart attack table?", "heart's electrical activity number of significant vessels heartAttack"]
4: ['I need to see the results of the resting ECG and how many major vessels are noted in each ca

In [ ]:
variations = df['response'].iloc[0]
original_query="SELECT restingelectrocardiographicrresults"
similarities = calculate_similarities(
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original query": variations["original_query"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

,Original query,Variation,Similarity
0,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,0.999999
1,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,ECG results count of major vessels heartAttack,0.807443
2,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,electrocardiographic results major blood vessels heartAttack,0.774385
3,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,heart's electrical activity number of significant vessels heartAttack,0.751739
4,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,cardiac electrical results vessel count heartAttack,0.767004
5,restingelectrocardiographicrresults numberofmajorvvessels heartAttack,resting ECG major vessel numbers heartAttack,0.797391


### Spider Style

In [ ]:
#@title spider style queries
import os
import openai
import pandas as pd
import logging
import json
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Define the OpenAICaller class
class OpenAICaller:
    def __init__(self, model_name, temperature=None, api_key=None, organization=None):
        self._client = None
        self.api_key = api_key if api_key is not None else os.getenv('OPENAI_API_KEY')
        self.model_name = model_name
        self.organization = organization if organization is not None else os.getenv('OPENAI_API_ORG')
        self.temperature = temperature

    @property
    def client(self):
        if self._client is None:
            self._client = OpenAI(
                organization=self.organization,
                api_key=self.api_key,
            )
        return self._client

    @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
    def get_embedding(self, text: str, model="text-embedding-3-small") -> list[float]:
        return self.client.embeddings.create(input=[text], model=model).data[0].embedding
    @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
    def get_chat_completion(self, messages: list[dict]):
        response = self.client.chat.completions.create(model=self.model_name,
                                                       messages=messages,
                                                       temperature=self.temperature,
                                                       seed=2024, response_format={"type": "json_object"})
        logging.info(f"Num of tokens processed: {response.usage.total_tokens}")
        return response.choices[0].message.content

# Define the function to generate variations
def generate_question_variations(structured_data, temperature, model_name="gpt-3.5-turbo"):
    schema_str = " | ".join(structured_data['schema'])
    messages = [
        {
            "role": "system",
            "content": (

        "You are a helpful assistant that assists the user in generating variations of the question."
        " Given a  SQL query, schema, table name,  and a SQL Tag containing operation, and condition (if any),"
        " return at least 5 possible question variations in natural language of the sql query in JSON format."
        " Do not use the exact words in the query such as columns or other specific terms; separate them if they are stuck together. It has to be natural, informal language where we simulate that we don't really know the labels but have a vague idea, just like the way a human would formulate the questions without referencing the exact same labels but something close to them."
        " Each item in the JSON output should be a dictionary with the key as the question number and the value as a list containing two elements:"
        " the first element should be the natural language question, and the second element should be a minimalistic description."
        " The minimalistic description should follow strictly the following template: operations (there could be many), target columns (there could be many), target table."
        " The minimalistic description should only use words from the original question."
        " The minimalistic description shouldn't contain filler words because it is used in embedding cosine similarity, so a single word can skew the similarity; be mindful."
        " The minimalistic description and the generated natural language questions should only use words from the original question except for alphanumeric and non-letter characters in words which should be transcribed to English."

        ##comment the following line if yoy wnat to generate spider style stuff

        " Assume the user may not be familiar with the exact labels or schema details, and may phrase things in an informal or imprecise way."
        " Use synonyms and alternate phrasings to reflect a variety of ways the user might ask the question without knowing the technical terms."
        " Do not explicitly mention SQL clauses like 'GROUP BY' or 'AVG'."
        " Change the order of the sentence and be creative with the phrasing."
        " Each generated question must be unique and structured differently from the others."
        # " The generated questions must be traceable back to the original SQL query."##added


        # " You have to follow the style of the following  natural language questions from the spider dataset because we want to have the same distribution:"
#         """
# Query: SELECT count(*) FROM head WHERE age  >  56, Question: How many heads of the departments are older than 56 ?
# Query: SELECT name ,  born_state ,  age FROM head ORDER BY age, Question: List the name, born state and age of the heads of departments ordered by age.
# Query: SELECT creation ,  name ,  budget_in_billions FROM department, Question: List the creation year, name and budget of each department.
# Query: SELECT name FROM head WHERE born_state != 'California', Question: What are the names of the heads who are born outside the California state?
# Query: SELECT born_state FROM head GROUP BY born_state HAVING count(*)  >=  3, Question: What are the names of the states where at least 3 heads were born?
# Query: SELECT count(DISTINCT temporary_acting) FROM management, Question: How many acting statuses are there?
# Query: SELECT Total_Horses FROM farm ORDER BY Total_Horses ASC, Question: List the total number of horses on farms in ascending order.
# Query: SELECT Total_Horses FROM farm ORDER BY Total_Horses ASC, Question: What is the total horses record for each farm, sorted ascending?
# Query: SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens', Question: What are the hosts of competitions whose theme is not "Aliens"?
# Query: SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens', Question: Return the hosts of competitions for which the theme is not Aliens?
# Query: SELECT Theme FROM farm_competition ORDER BY YEAR ASC, Question: What are the themes of farm competitions sorted by year in ascending order?
# Query: SELECT Theme FROM farm_competition ORDER BY YEAR ASC, Question: Return the themes of farm competitions, sorted by year ascending.
# Query: SELECT count(DISTINCT Status) FROM city, Question: How many different statuses do cities have?
# Query: SELECT count(DISTINCT Status) FROM city, Question: Count the number of different statuses.
# Query: SELECT Official_Name FROM city ORDER BY Population DESC, Question: List official names of cities in descending order of population.
# Query: SELECT Official_Name FROM city ORDER BY Population DESC, Question: What are the official names of cities, ordered descending by population?
# Query: SELECT Status FROM city GROUP BY Status ORDER BY COUNT(*) ASC, Question: Please show the different statuses, ordered by the number of cities that have each.
# Query: SELECT Status FROM city GROUP BY Status ORDER BY COUNT(*) ASC, Question: Return the different statuses of cities, ascending by frequency.
# Query: SELECT Official_Name FROM city WHERE Population  >  1500 OR Population  <  500, Question: Find the official names of cities with population bigger than 1500 or smaller than 500.
# Query: SELECT Official_Name FROM city WHERE Population  >  1500 OR Population  <  500, Question: What are the official names of cities that have population over 1500 or less than 500?
# Query: SELECT Census_Ranking FROM city WHERE Status !=  "Village", Question: Show the census ranking of cities whose status are not "Village".
# Query: SELECT Census_Ranking FROM city WHERE Status !=  "Village", Question: What are the census rankings of cities that do not have the status "Village"?
# Query: SELECT student_id FROM student_course_attendance, Question: List the id of students who attended some courses?
# Query: SELECT student_id FROM student_course_attendance, Question: What are the ids of all students who have attended at least one course?
# Query: SELECT candidate_id FROM candidate_assessments WHERE asessment_outcome_code = "Pass", Question: Find id of candidates whose assessment code is "Pass"?
# Query: SELECT candidate_id FROM candidate_assessments WHERE asessment_outcome_code = "Pass", Question: What are the ids of the candidates that have an outcome code of Pass?
# Query: SELECT student_id FROM student_course_attendance WHERE course_id  =  301, Question: What are the id of students who registered course 301?
# Query: SELECT student_id FROM student_course_attendance WHERE course_id  =  301, Question: What are the ids of the students who registered for course 301?
# Query: SELECT course_name FROM courses ORDER BY course_name, Question: List the names of courses in alphabetical order?
# Query: SELECT course_name FROM courses ORDER BY course_name, Question: What are the names of the courses in alphabetical order?
# Query: SELECT first_name FROM people ORDER BY first_name, Question: List the first names of people in alphabetical order?
#         """
#                 """
# Query: SELECT count(*) FROM head WHERE age  >  56, Question: How many heads of the departments are older than 56 ?
# Query: SELECT name ,  born_state ,  age FROM head ORDER BY age, Question: List the name, born state and age of the heads of departments ordered by age.
# Query: SELECT creation ,  name ,  budget_in_billions FROM department, Question: List the creation year, name and budget of each department.
# Query: SELECT name FROM head WHERE born_state != 'California', Question: What are the names of the heads who are born outside the California state?
# Query: SELECT born_state FROM head GROUP BY born_state HAVING count(*)  >=  3, Question: What are the names of the states where at least 3 heads were born?
# Query: SELECT count(DISTINCT temporary_acting) FROM management, Question: How many acting statuses are there?
# Query: SELECT Total_Horses FROM farm ORDER BY Total_Horses ASC, Question: List the total number of horses on farms in ascending order.
# Query: SELECT Total_Horses FROM farm ORDER BY Total_Horses ASC, Question: What is the total horses record for each farm, sorted ascending?
# Query: SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens', Question: What are the hosts of competitions whose theme is not "Aliens"?
# Query: SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens', Question: Return the hosts of competitions for which the theme is not Aliens?
# Query: SELECT Theme FROM farm_competition ORDER BY YEAR ASC, Question: What are the themes of farm competitions sorted by year in ascending order?
# Query: SELECT Theme FROM farm_competition ORDER BY YEAR ASC, Question: Return the themes of farm competitions, sorted by year ascending.
# Query: SELECT count(DISTINCT Status) FROM city, Question: How many different statuses do cities have?
# Query: SELECT count(DISTINCT Status) FROM city, Question: Count the number of different statuses.
# Query: SELECT Official_Name FROM city ORDER BY Population DESC, Question: List official names of cities in descending order of population.
#         """
            )
        },
        {
            "role": "user",
            "content": """\nQuery: SELECT AVG(temperature) FROM weather_data GROUP BY city;
                        \nSchema: city, temperature\nTable_name: weather_data\nSQL-TAG: GROUPBY-AVG"""
        },
        {
            "role": "system",
            "content": json.dumps({
                "original_query": ["SELECT AVG(temperature) FROM weather_data GROUP BY city;", "average temperature group by city weather data"],
                "1": ["What is the mean temeprature in the diffrent cities in the weather data table?", "mean temperature in cities weather data"],
                "2": ["Using the weather_data, What is the usual climate in terms of temperature for each of the urban areas", "usual temperature for each of urban areas weather data"],
                "3": ["What are the typical thermal conditions in each of these cities that you can find in the weather data table?", "typical thermal conditions in each of cities weather data"],
                "4": ["Could you tell me the average degree of warmth in each of the cities based on the weather data table?", "mean temperature in each of cities weather data"],
                "5": ["What is the norm in terms of heat level in each of the cities from the weather data table?", "norm heat level in each of cities weather data"]
            })
        },
        {
            "role": "user",
            "content": f"Query: {structured_data['query']}\nSchema: {', '.join(structured_data['schema'])}\nTable_name: {structured_data['table_name']}\SQL-TAG: {structured_data['SQL-TAG']}"
        }
    ]

    # Initialize OpenAICaller and get the response
    openai_caller = OpenAICaller(model_name, temperature)
    response = openai_caller.get_chat_completion(messages)

    try:
        response_dict = json.loads(response)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        response_dict = {}

    # Save to JSON and create dataframe
    df = pd.DataFrame({
        'schema': [schema_str],
        'response': [response_dict],
        'query': [structured_data['query']],
        # 'minimal_description': [minimal_description]
    })

    return df,response_dict

# Define function to calculate similarity between original and generated questions
def calculate_similarities(variations, model_name="text-embedding-3-large"):
    openai_caller = OpenAICaller(model_name)
    original_embedding = openai_caller.get_embedding(variations["original_query"][1], model_name)
    similarities = {}
    for key, variation in variations.items():
        variation_embedding = openai_caller.get_embedding(variation[1], model_name)  # Using only the minimal desciption part
        similarity = cosine_similarity(
            np.array(original_embedding),
            np.array(variation_embedding)
        )
        similarities[key] = similarity

    return similarities


In [ ]:
#@title spider style queries
import os
import openai
import pandas as pd
import logging
import json
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Define the OpenAICaller class
class OpenAICaller:
    def __init__(self, model_name, temperature=None, api_key=None, organization=None):
        self._client = None
        self.api_key = api_key if api_key is not None else os.getenv('OPENAI_API_KEY')
        self.model_name = model_name
        self.organization = organization if organization is not None else os.getenv('OPENAI_API_ORG')
        self.temperature = temperature

    @property
    def client(self):
        if self._client is None:
            self._client = OpenAI(
                organization=self.organization,
                api_key=self.api_key,
            )
        return self._client

    @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
    def get_embedding(self, text: str, model="text-embedding-3-small") -> list[float]:
        return self.client.embeddings.create(input=[text], model=model).data[0].embedding
    @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
    def get_chat_completion(self, messages: list[dict]):
        response = self.client.chat.completions.create(model=self.model_name,
                                                       messages=messages,
                                                       temperature=self.temperature,
                                                       seed=2024, response_format={"type": "json_object"})
        logging.info(f"Num of tokens processed: {response.usage.total_tokens}")
        return response.choices[0].message.content

# Define the function to generate variations
def generate_question_variations(structured_data, temperature, model_name="gpt-3.5-turbo"):
    schema_str = " | ".join(structured_data['schema'])
    messages = [
        {
            "role": "system",
            "content": (

        "You are a helpful assistant that assists the user in generating variations of the question."
        " Given a  SQL query, schema, table name,  and a SQL Tag containing operation, and condition (if any),"
        " return at least 5 possible question variations in natural language of the sql query in JSON format."
        " Do not use the exact words in the query such as columns or other specific terms; separate them if they are stuck together. It has to be natural, informal language where we simulate that we don't really know the labels but have a vague idea, just like the way a human would formulate the questions without referencing the exact same labels but something close to them."
        " Each item in the JSON output should be a dictionary with the key as the question number and the value as a list containing two elements:"
        " the first element should be the natural language question, and the second element should be a minimalistic description."
        " The minimalistic description should follow strictly the following template: operations (there could be many), target columns (there could be many), target table."
        " The minimalistic description should only use words from the original question."
        " The minimalistic description shouldn't contain filler words because it is used in embedding cosine similarity, so a single word can skew the similarity; be mindful."
        " The minimalistic description and the generated natural language questions should only use words from the original question except for alphanumeric and non-letter characters in words which should be transcribed to English."

        ##comment the following line if you wnat to generate spider style stuff

        " Assume the user may not be familiar with the exact labels or schema details, and may phrase things in an informal or imprecise way."
        " Use synonyms and alternate phrasings to reflect a variety of ways the user might ask the question without knowing the technical terms."
        " Do not explicitly mention SQL clauses like 'GROUP BY' or 'AVG'."
        " Change the order of the sentence and be creative with the phrasing."
        " Each generated question must be unique and structured differently from the others."
        " The generated questions must be traceable back to the original SQL query."##added


#         " You have to follow the style of the following  natural language questions from the spider dataset because we want to have the same distribution:"
#                 """
# Query: SELECT count(*) FROM head WHERE age  >  56, Question: How many heads of the departments are older than 56 ?
# Query: SELECT name ,  born_state ,  age FROM head ORDER BY age, Question: List the name, born state and age of the heads of departments ordered by age.
# Query: SELECT creation ,  name ,  budget_in_billions FROM department, Question: List the creation year, name and budget of each department.
# Query: SELECT name FROM head WHERE born_state != 'California', Question: What are the names of the heads who are born outside the California state?
# Query: SELECT born_state FROM head GROUP BY born_state HAVING count(*)  >=  3, Question: What are the names of the states where at least 3 heads were born?
# Query: SELECT count(DISTINCT temporary_acting) FROM management, Question: How many acting statuses are there?
# Query: SELECT Total_Horses FROM farm ORDER BY Total_Horses ASC, Question: List the total number of horses on farms in ascending order.
# Query: SELECT Total_Horses FROM farm ORDER BY Total_Horses ASC, Question: What is the total horses record for each farm, sorted ascending?
#  """
# Query: SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens', Question: What are the hosts of competitions whose theme is not "Aliens"?
# Query: SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens', Question: Return the hosts of competitions for which the theme is not Aliens?
# Query: SELECT Theme FROM farm_competition ORDER BY YEAR ASC, Question: What are the themes of farm competitions sorted by year in ascending order?
# Query: SELECT Theme FROM farm_competition ORDER BY YEAR ASC, Question: Return the themes of farm competitions, sorted by year ascending.
# Query: SELECT count(DISTINCT Status) FROM city, Question: How many different statuses do cities have?
# Query: SELECT count(DISTINCT Status) FROM city, Question: Count the number of different statuses.
# Query: SELECT Official_Name FROM city ORDER BY Population DESC, Question: List official names of cities in descending order of population.
        # """
            )
        },
        {
            "role": "user",
            "content": """\nQuery: SELECT AVG(temperature) FROM weather_data GROUP BY city;
                        \nSchema: city, temperature\nTable_name: weather_data\nSQL-TAG: GROUPBY-AVG"""
        },
        {
            "role": "system",
            "content": json.dumps({
                "original_query": ["SELECT AVG(temperature) FROM weather_data GROUP BY city;", "average temperature group by city weather data"],
                "1": ["What is the mean temeprature in the diffrent cities in the weather data table?", "mean temperature in cities weather data"],
                "2": ["Using the weather_data, What is the usual climate in terms of temperature for each of the urban areas", "usual temperature for each of urban areas weather data"],
                "3": ["What are the typical thermal conditions in each of these cities that you can find in the weather data table?", "typical thermal conditions in each of cities weather data"],
                "4": ["Could you tell me the average degree of warmth in each of the cities based on the weather data table?", "mean temperature in each of cities weather data"],
                "5": ["What is the norm in terms of heat level in each of the cities from the weather data table?", "norm heat level in each of cities weather data"]
            })
        },
        {
            "role": "user",
            "content": f"Query: {structured_data['query']}\nSchema: {', '.join(structured_data['schema'])}\nTable_name: {structured_data['table_name']}\SQL-TAG: {structured_data['SQL-TAG']}"
        }
    ]

    # Initialize OpenAICaller and get the response
    openai_caller = OpenAICaller(model_name, temperature)
    response = openai_caller.get_chat_completion(messages)

    try:
        response_dict = json.loads(response)
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        response_dict = {}

    # Save to JSON and create dataframe
    df = pd.DataFrame({
        'schema': [schema_str],
        'response': [response_dict],
        'query': [structured_data['query']],
        # 'minimal_description': [minimal_description]
    })

    return df,response_dict

# Define function to calculate similarity between original and generated questions
def calculate_similarities(variations, model_name="text-embedding-3-large"):
    openai_caller = OpenAICaller(model_name)
    original_embedding = openai_caller.get_embedding(variations["original_query"][1], model_name)
    similarities = {}
    for key, variation in variations.items():
        variation_embedding = openai_caller.get_embedding(variation[1], model_name)  # Using only the minimal desciption part
        similarity = cosine_similarity(
            np.array(original_embedding),
            np.array(variation_embedding)
        )
        similarities[key] = similarity

    return similarities


In [ ]:
# structured_input = {
#     'table_name': 'heartAttack',
#     'schema': ['age', 'sex', 'chestpaintype', 'restingbloodpressure',
#        'cholestoralinmg', 'fastingbloodsugar',
#        'restingelectrocardiographicrresults', 'numberofmajorvvessels', 'thall',
#        'output'],
#     # 'operation': 'SELECT',
#     # 'condition': 'None',
#     'SQL-TAG': 'SELECT-RANDOM-COL',
#     'query': 'SELECT restingelectrocardiographicrresults, numberofmajorvvessels FROM heartAttack',
#     # 'question': 'Show all restingelectrocardiographicrresults, numberofmajorvvessels in the table heartAttack'
# }
structured_input = {
    'table_name': 'breastCancer',
    'schema': ['patientidentifier', 'age', 'menopausalstatus', 'tumorsize',
       'tumorgrade', 'numberpositivelymphnodes', 'progesteronereceptor',
       'estrogenreceptor', 'hormonaltherapy', 'status'],
    # 'operation': 'MAX',
    # 'condition': 'None',
    'SQL-TAG': 'SIMPLE-AGG-MAX',
    'query': 'SELECT MAX(estrogenreceptor) FROM breastCancer;',
    # 'question': 'Find the maximum estrogenreceptor for the table breastCancer'
}

temeprature=0
model_name="gpt-4o"
df ,response_dict= generate_question_variations(structured_input,temeprature,model_name)
# Print the output
for index, row in df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    # print(row["question"])
    print(row["query"])

patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes | progesteronereceptor | estrogenreceptor | hormonaltherapy | status
original_query: ['SELECT MAX(estrogenreceptor) FROM breastCancer;', 'maximum estrogen receptor breast cancer']
1: ["What's the highest level of estrogen receptor found in the breast cancer data?", 'highest estrogen receptor breast cancer']
2: ['Can you tell me the peak estrogen receptor value in the breast cancer records?', 'peak estrogen receptor breast cancer']
3: ['What is the top estrogen receptor measurement in the breast cancer dataset?', 'top estrogen receptor breast cancer']
4: ["What's the maximum estrogen receptor reading in the breast cancer information?", 'maximum estrogen receptor breast cancer']
5: ['Could you find the greatest estrogen receptor level in the breast cancer data?', 'greatest estrogen receptor breast cancer']
SELECT MAX(estrogenreceptor) FROM breastCancer;


In [ ]:
variations = df['response'].iloc[0]
original_query="SELECT restingelectrocardiographicrresults"
similarities = calculate_similarities(
    variations=variations,
    model_name="text-embedding-3-large"
)
display_data = []
for key, variation in variations.items():
    similarity = similarities.get(key, None)
    display_data.append({
        "Original query": variations["original_query"][1],
        "Variation": variation[1],
        "Similarity": similarity
    })

# Convert to DataFrame
similarities_df = pd.DataFrame(display_data)

# Display the DataFrame
similarities_df

In [ ]:
#@title old
import pandas as pd

df=cleaned_df.copy()
group_counts = df['sql_tags_trunc'].value_counts()
# print(f"group_counts: {group_counts}")
# print("###############################")

group_proportions = group_counts / len(df)
# print(f"group_proportions: {group_proportions}")
# print("###############################")

num_samples = 21  # Specify the desired number of queries here

inverted_prob=1/group_proportions
inverted_sum=inverted_prob.sum()
inverted_prob_norm=inverted_prob/inverted_sum
# print(f"inverted_prob_norm: {inverted_prob_norm}")
# print("###############################")

group_samples = (num_samples *inverted_prob_norm).round().astype(int)
# print(f"group_samples: {group_samples}")

# group_samples = group_samples.clip(lower=1)
print(f"group_samples: {group_samples}")

sampled_queries = []
for tag, count in group_samples.items():
    sampled_queries.extend(df[df['sql_tags_trunc'] == tag].sample(n=count, random_state=137,replace=False).index.tolist())##prev 45

# Create a new dataframe to store the generated variations
variations_df = pd.DataFrame(columns=['index', 'schema', 'response', 'query'])

# Iterate over the sampled queries and generate variations
for index in sampled_queries:
    query_row = df.loc[index]

    structured_input = {
        'table_name': query_row['table_used'],
        'schema': query_row['schema'],
        'SQL-TAG': query_row['sql_tags'],
        'query': query_row['query']
    }

    temperature = 0.1
    model_name = "gpt-4-turbo"

    variations = generate_question_variations(structured_input, temperature, model_name)

    # Create a new DataFrame with the row to append
    new_row = pd.DataFrame([{
      'index': index,
      'schema': variations['schema'][0],
      'response': variations['response'][0],
      'query': variations['query'][0]
    }])

    # Append the new row to variations_df using pd.concat
    variations_df = pd.concat([variations_df, new_row], ignore_index=True)

# Print the output
for _, row in variations_df.iterrows():
    print(row["schema"])
    response_dict = row["response"]
    for key, value in response_dict.items():
        print(f"{key}: {value}")
    print(row["query"])

In [ ]:
variations_df

,index,schema,response,query
0,974,patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes |...,"{'original_query': ['SELECT status, MIN(tumorgrade) FROM breastCancer GROUP BY status', 'minimum...","SELECT status, MIN(tumorgrade) FROM breastCancer GROUP BY status"
1,371,patientidentifier | age | menopausalstatus | tumorsize | tumorgrade | numberpositivelymphnodes |...,"{'original_query': ['SELECT DISTINCT hormonaltherapy, status, menopausalstatus FROM breastCancer...","SELECT DISTINCT hormonaltherapy, status, menopausalstatus FROM breastCancer"


In [ ]:
sampled_queries

[]

In [ ]:
print(len(df))

429


In [ ]:
group_counts = df['sql_tags_trunc'].value_counts()
group_counts

sql_tags_trunc
HAVING      186
WHERE        96
SELECT       40
ORDERBY      32
SIMPLE       31
GROUPBY      28
DISTINCT     16
Name: count, dtype: int64

In [ ]:
inverted_prob=1/group_proportions
inverted_sum=inverted_prob.sum()
inverted_prob_norm=inverted_prob/inverted_sum
inverted_prob_norm

sql_tags_trunc
HAVING      0.026548
WHERE       0.051436
SELECT      0.123447
ORDERBY     0.154309
SIMPLE      0.159287
GROUPBY     0.176353
DISTINCT    0.308619
Name: count, dtype: float64

In [ ]:
group_proportions/group_proportions*1/7

sql_tags_trunc
HAVING      0.142857
WHERE       0.142857
SELECT      0.142857
ORDERBY     0.142857
SIMPLE      0.142857
GROUPBY     0.142857
DISTINCT    0.142857
Name: count, dtype: float64

### Split before and only perform the reformulation on the trainset.

In [ ]:
percentage_valid = 0.3
temp_indices = cleaned_df.groupby('sql_tags_trunc', group_keys=False).apply(lambda x: x.sample(frac=percentage_valid, random_state=42)).index
temp_df = cleaned_df.loc[temp_indices]
train_df_full = cleaned_df.drop(temp_indices)

# Check the size of each set to confirm the split
print(f"Total dataset size: {cleaned_df.shape[0]}")
print(f"Training set size: {train_df_full.shape[0]}")
print(f"Tempset size: {temp_df.shape[0]}")

Total dataset size: 429
Training set size: 300
Tempset size: 129


In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
percentage_test = 0.5
test_indices = temp_df.groupby('sql_tags_trunc', group_keys=False).apply(lambda x: x.sample(frac=percentage_test, random_state=42)).index
test_df = cleaned_df.loc[test_indices]
valid_df = temp_df.drop(test_indices)

# Check the size of each set to confirm the split
print(f"Total dataset size: {temp_df.shape[0]}")
print(f"validation set size: {valid_df.shape[0]}")
print(f"test size: {test_df.shape[0]}")

Total dataset size: 129
validation set size: 66
test size: 63


In [ ]:
import pandas as pd
import numpy as np
df = train_df_full.copy()
group_counts = df['sql_tags_trunc'].value_counts()
group_proportions = group_counts / len(df)

num_samples = 110  # Specify the desired number of queries here

# inverted_prob = 1 / group_proportions
# inverted_sum = inverted_prob.sum()
# inverted_prob_norm = inverted_prob / inverted_sum

##uniform distribution
inverted_prob_norm= group_proportions/group_proportions*1/7

group_samples = (num_samples * inverted_prob_norm).round().astype(int)
group_samples = group_samples.clip(lower=1)
print(f"group_samples: {group_samples}")
sampled_queries = []
# for tag, count in group_samples.items():
#     sampled_queries.extend(df[df['sql_tags_trunc'] == tag].sample(n=count,random_state=137 ,replace=False).index.tolist())

for tag, count in group_samples.items():
    available_samples = df[df['sql_tags_trunc'] == tag]
    if len(available_samples) < count:
      print(f"sampled with replacment tag: {tag}, count : {count}")
      sampled_queries.extend(
            available_samples.sample(n=count, random_state=137, replace=True).index.tolist()
        )
    else:
        sampled_queries.extend(
            available_samples.sample(n=count, random_state=137, replace=False).index.tolist()
        )

# Now, `sampled_queries` will contain the sampled indices with the required logic.



# Create a new dataframe to store the generated variations
variations_df = pd.DataFrame(columns=['index', 'schema', 'minimalistic_description', 'variation'])

# Iterate over the sampled queries and generate variations
counter=0
for index in sampled_queries:
    print(f"counter: {counter}\n")
    query_row = df.loc[index]

    structured_input = {
        'table_name': query_row['table_used'],
        'schema': query_row['schema'],
        'SQL-TAG': query_row['sql_tags'],
        'query': query_row['query']
    }

    temperature = 0
    model_name = "gpt-4o"

    variations,response_dict = generate_question_variations(structured_input, temperature, model_name)

    # Extract the original query and variations from the response dictionary
    #
    try:
        original_query_Min_Descri = response_dict['original_query'][1]
    except KeyError:
        original_query_Min_Descri = np.nan
        print(f"No original_query_Min_Descri in the response for index{index} and counter {counter}")

    for i in range(1, 6):
        variation = response_dict[str(i)][0]
        minimalistic_description = response_dict[str(i)][1]

        # Create a new row with the extracted information
        new_row = pd.DataFrame([{
            'index': index,
            'schema': query_row['schema'],
            'minimalistic_description': minimalistic_description,
            'variation': variation,
            'original_query': query_row['query'],
            'original_query_Min_Descri': original_query_Min_Descri
        }])

        # Append the new row to variations_df using pd.concat
        variations_df = pd.concat([variations_df, new_row], ignore_index=True)
    counter+=1

# Print the output
for _, row in variations_df.iterrows():
    print(f"Schema: {row['schema']}")
    print(f"Minimalistic Description: {row['minimalistic_description']}")
    print(f"Variation: {row['variation']}")
    print()



group_samples: sql_tags_trunc
HAVING      16
WHERE       16
SELECT      16
ORDERBY     16
SIMPLE      16
GROUPBY     16
DISTINCT    16
Name: count, dtype: int64
sampled with replacment tag: DISTINCT, count : 16
counter: 0

counter: 1

counter: 2

counter: 3

counter: 4

counter: 5

counter: 6

counter: 7

counter: 8

counter: 9

counter: 10

counter: 11

counter: 12

counter: 13

counter: 14

counter: 15

counter: 16

counter: 17

counter: 18

counter: 19

counter: 20

counter: 21

counter: 22

counter: 23

counter: 24

counter: 25

counter: 26

counter: 27

counter: 28

counter: 29

counter: 30

counter: 31

counter: 32

counter: 33

counter: 34

counter: 35

counter: 36

counter: 37

counter: 38

counter: 39

counter: 40

counter: 41

counter: 42

counter: 43

counter: 44

counter: 45

counter: 46

counter: 47

counter: 48

counter: 49

counter: 50

counter: 51

counter: 52

counter: 53

counter: 54

counter: 55

counter: 56

counter: 57

counter: 58

counter: 59

counter: 60

counte

In [ ]:
variations_df

,index,schema,minimalistic_description,variation,original_query,original_query_Min_Descri
0,1565,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",fasting blood sugar levels average number of major vessels heart attack,What are the fasting blood sugar levels where the average number of major vessels is at least 0....,SELECT fastingbloodsugar FROM heartAttack GROUP BY fastingbloodsugar HAVING AVG(numberofmajorvve...,fasting blood sugar group by fasting blood sugar having average number of major vessels greater ...
1,1565,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",fasting blood sugar values mean number of major vessels heart attack,Can you list the fasting blood sugar values for cases where the mean number of major vessels is ...,SELECT fastingbloodsugar FROM heartAttack GROUP BY fastingbloodsugar HAVING AVG(numberofmajorvve...,fasting blood sugar group by fasting blood sugar having average number of major vessels greater ...
2,1565,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",fasting blood sugar readings average major vessels heart attack,Which fasting blood sugar readings correspond to an average of 0.38 or more major vessels in the...,SELECT fastingbloodsugar FROM heartAttack GROUP BY fastingbloodsugar HAVING AVG(numberofmajorvve...,fasting blood sugar group by fasting blood sugar having average number of major vessels greater ...
3,1565,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",fasting blood sugar levels average major vessels heart attack,"In the heart attack data, what fasting blood sugar levels have an average number of major vessel...",SELECT fastingbloodsugar FROM heartAttack GROUP BY fastingbloodsugar HAVING AVG(numberofmajorvve...,fasting blood sugar group by fasting blood sugar having average number of major vessels greater ...
4,1565,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",fasting blood sugar levels average number of major vessels heart attack,What are the fasting blood sugar levels in the heart attack table where the average number of ma...,SELECT fastingbloodsugar FROM heartAttack GROUP BY fastingbloodsugar HAVING AVG(numberofmajorvve...,fasting blood sugar group by fasting blood sugar having average number of major vessels greater ...
...,...,...,...,...,...,...
555,366,"[patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...",distinct menopausal statuses breast cancer,What are the different menopausal statuses listed in the breast cancer data?,SELECT DISTINCT menopausalstatus FROM breastCancer,distinct menopausal status breast cancer
556,366,"[patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...",unique menopausal conditions breast cancer,Can you show me the unique menopausal conditions found in the breast cancer records?,SELECT DISTINCT menopausalstatus FROM breastCancer,distinct menopausal status breast cancer
557,366,"[patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...",various menopausal states breast cancer,What are the various menopausal states recorded in the breast cancer dataset?,SELECT DISTINCT menopausalstatus FROM breastCancer,distinct menopausal status breast cancer
558,366,"[patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...",distinct menopausal categories breast cancer,Could you list the distinct menopausal categories in the breast cancer table?,SELECT DISTINCT menopausalstatus FROM breastCancer,distinct menopausal status breast cancer


In [ ]:
variations_df.columns

Index(['index', 'schema', 'minimalistic_description', 'variation',
       'original_query', 'original_query_Min_Descri'],
      dtype='object')

In [ ]:
cleaned_df.columns

Index(['ID', 'answer_text', 'db_id', 'table_used', 'sql_tags', 'query',
       'question', 'domain', 'operator', 'answer_coordinates', 'float_value',
       'schema', 'sql_tags_trunc'],
      dtype='object')

In [ ]:
# Creating DataFrames
# original_df = cleaned_df.copy()
original_df = train_df_full.copy()
varieted_df = variations_df.copy()
varieted_df=varieted_df.drop(columns=["schema"])

# Merging the DataFrames on ID and index
merged_df = pd.merge(original_df, varieted_df, left_on=['ID'], right_on=['index'])

# Creating the new DataFrame
new_df = merged_df.drop(columns=['index', 'minimalistic_description', 'variation',
                                 'original_query', 'original_query_Min_Descri'])

new_df['question'] = merged_df['variation']


In [ ]:
variations_df.to_pickle('/content/drive/MyDrive/MakingRequestsTo_GPT_data/gpt4o-110_examples_temp_0_variations_df.pkl')

In [ ]:
new_df.to_pickle('/content/drive/MyDrive/MakingRequestsTo_GPT_data/QATCH-varieted_data_ready_gpt_4_turbo_25exples_point1Temp.pkl')

In [ ]:
new_df["sql_tags_trunc"].value_counts()

sql_tags_trunc
SELECT      80
ORDERBY     80
DISTINCT    80
WHERE       80
GROUPBY     80
HAVING      80
SIMPLE      80
Name: count, dtype: int64

In [ ]:
new_df

,ID,answer_text,db_id,table_used,sql_tags,query,question,domain,operator,answer_coordinates,float_value,schema,sql_tags_trunc
0,116,"[62, male, typicalAngina, 140, 268, false, normal, 2, 2, noHeartAttack, 64, female, asymptomatic...",propiretary4,heartAttack,SELECT-ALL,SELECT * FROM heartAttack,Can you show me everything in the heart attack dataset?,medicine,NaN,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1),...",NaN,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",SELECT
1,116,"[62, male, typicalAngina, 140, 268, false, normal, 2, 2, noHeartAttack, 64, female, asymptomatic...",propiretary4,heartAttack,SELECT-ALL,SELECT * FROM heartAttack,"I need all the details from the heart attack table, can you get that?",medicine,NaN,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1),...",NaN,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",SELECT
2,116,"[62, male, typicalAngina, 140, 268, false, normal, 2, 2, noHeartAttack, 64, female, asymptomatic...",propiretary4,heartAttack,SELECT-ALL,SELECT * FROM heartAttack,Could you provide the full information from the heart attack data?,medicine,NaN,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1),...",NaN,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",SELECT
3,116,"[62, male, typicalAngina, 140, 268, false, normal, 2, 2, noHeartAttack, 64, female, asymptomatic...",propiretary4,heartAttack,SELECT-ALL,SELECT * FROM heartAttack,What are all the records in the heart attack dataset?,medicine,NaN,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1),...",NaN,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",SELECT
4,116,"[62, male, typicalAngina, 140, 268, false, normal, 2, 2, noHeartAttack, 64, female, asymptomatic...",propiretary4,heartAttack,SELECT-ALL,SELECT * FROM heartAttack,Can you give me the complete data from the heart attack table?,medicine,NaN,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1),...",NaN,"[age, sex, chestpaintype, restingbloodpressure, cholestoralinmg, fastingbloodsugar, restingelect...",SELECT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,1846,[0],propiretary4,breastCancer,SIMPLE-AGG-MIN,SELECT MIN(estrogenreceptor) FROM breastCancer,What's the lowest level of estrogen receptor found in the breast cancer data?,medicine,NaN,"[(6, 6)]",0.0,"[patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...",SIMPLE
556,1846,[0],propiretary4,breastCancer,SIMPLE-AGG-MIN,SELECT MIN(estrogenreceptor) FROM breastCancer,Can you tell me the smallest amount of estrogen receptor recorded in the breast cancer dataset?,medicine,NaN,"[(6, 6)]",0.0,"[patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...",SIMPLE
557,1846,[0],propiretary4,breastCancer,SIMPLE-AGG-MIN,SELECT MIN(estrogenreceptor) FROM breastCancer,What is the minimum value of estrogen receptor in the breast cancer table?,medicine,NaN,"[(6, 6)]",0.0,"[patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...",SIMPLE
558,1846,[0],propiretary4,breastCancer,SIMPLE-AGG-MIN,SELECT MIN(estrogenreceptor) FROM breastCancer,What's the least estrogen receptor level noted in the breast cancer records?,medicine,NaN,"[(6, 6)]",0.0,"[patientidentifier, age, menopausalstatus, tumorsize, tumorgrade, numberpositivelymphnodes, prog...",SIMPLE
